In [1]:
model_used = "base"

# Paths

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"


In [3]:
use_template = True if model_used=='chat' else False
use_template

False

In [4]:
hf_llama_path = "/data/home/milinbhade/LLaMa/hf_llama/"
hf_llama_chat_path = "/data/home/milinbhade/LLaMa/hf_llama_chat/"

alpacha_dataset_path = "/data/home/milinbhade/LLaMa/pretrain_data/alpaca/alpaca-cleaned-dataset"

# Utilities

In [5]:
import numpy as np
import random
import torch
from datasets import load_dataset
import fnmatch
from tqdm import tqdm
import torch.nn as nn
import time
import logging

import json
import transformers

In [6]:
torch.cuda.device_count()

1

In [7]:
torch.cuda.is_available()

True

In [8]:
from transformers import LlamaTokenizer, AutoTokenizer, AutoModelForCausalLM

## Load Dataset

In [9]:
class TokenizerWrapper:
    def __init__(self, input_ids):
        self.input_ids = input_ids

# Load and process wikitext2 dataset
def get_wikitext2(nsamples, seed, seqlen, tokenizer):
    # Load train and test datasets
    traindata = load_dataset('wikitext', 'wikitext-2-raw-v1', split='train')
    valdata = load_dataset('wikitext', 'wikitext-2-raw-v1', split='validation')
    testdata = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')

    # Encode datasets
    trainenc = tokenizer(" ".join(traindata['text']), return_tensors='pt')
    valenc = tokenizer("\n\n".join(valdata['text']), return_tensors='pt')
    testenc = tokenizer("\n\n".join(testdata['text']), return_tensors='pt')

    # Generate samples from training set
    random.seed(seed)
    trainloader = []
    for _ in range(nsamples):
        i = random.randint(0, trainenc.input_ids.shape[1] - seqlen - 1)
        j = i + seqlen
        inp = trainenc.input_ids[:, i:j]
        tar = inp.clone()
        tar[:, :-1] = -100
        trainloader.append((inp, tar))
    return trainloader, valenc, testenc

def get_loaders(name, nsamples=128, seed=0, seqlen=2048, tokenizer=None):
    if 'wikitext2' in name:
        return get_wikitext2(nsamples, seed, seqlen, tokenizer)
    if "c4" in name:
        return get_c4(nsamples, seed, seqlen, tokenizer)
    if "bookcorpus" in name:
        return get_book_corpus(nsamples, seed, seqlen, tokenizer)
    if "ptb" in name:
        return get_ptb(nsamples, seed, seqlen, tokenizer)

## Eval PPL on wikitext

In [10]:
# Function to evaluate perplexity (ppl) on a specified model and tokenizer
def eval_ppl(model, tokenizer, device=torch.device("cuda:0")):
    # Set dataset
    dataset = "wikitext2"

    # Print status
    print(f"evaluating on {dataset}")

    # Get the test loader
    _, _, testloader = get_loaders(
        dataset, seed=0, seqlen=model.seqlen, tokenizer=tokenizer 
    )

    # Evaluate ppl in no grad context to avoid updating the model
    with torch.no_grad():
        ppl_test = eval_ppl_wikitext(model, testloader, 1, device)
    return ppl_test 

# Function to evaluate perplexity (ppl) specifically on the wikitext dataset
def eval_ppl_wikitext_train(model, trainloader, bs=1, device=None):
    # Get input IDs
    # testenc = testenc.input_ids

    # Calculate number of samples
    # nsamples = testenc.numel() // model.seqlen
    nsamples = len(trainloader)

    # List to store negative log likelihoods
    nlls = []
    print(f"nsamples {nsamples}")

    # Loop through each batch
    for i in range(0,nsamples,bs):
        if i % 50 == 0:
            print(f"sample {i}")

        # Calculate end index
        j = min(i+bs, nsamples)

        # Prepare inputs and move to device
        # inputs = testenc[:,(i * model.seqlen):(j * model.seqlen)].to(device)
        inputs = trainloader[i][0].to(device)
        inputs = inputs.reshape(j-i, model.seqlen)

        # Forward pass through the model
        lm_logits = model(inputs).logits

        # Shift logits and labels for next token prediction
        shift_logits = lm_logits[:, :-1, :].contiguous()
        shift_labels = inputs[:, 1:]

        # Compute loss
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(shift_logits.reshape(-1, shift_logits.size(-1)), shift_labels.reshape(-1))

        # Calculate negative log likelihood
        neg_log_likelihood = loss.float() * model.seqlen * (j-i)

        # Append to list of negative log likelihoods
        nlls.append(neg_log_likelihood)

    # Compute perplexity
    ppl = torch.exp(torch.stack(nlls).sum() / (nsamples * model.seqlen))

    # Empty CUDA cache to save memory
    torch.cuda.empty_cache()

    return ppl.item()

# Function to evaluate perplexity (ppl) specifically on the wikitext dataset
def eval_ppl_wikitext(model, testenc, bs=1, device=None):
    # Get input IDs
    testenc = testenc.input_ids

    # Calculate number of samples
    # nsamples = testenc.numel() // model.seqlen
    nsamples = 25

    # List to store negative log likelihoods
    nlls = []
    print(f"nsamples {nsamples}")

    # Loop through each batch
    for i in tqdm(range(0,nsamples,bs), desc ="WikiText Validation: "):
#         if i % 5 == 0:
#             print(f"sample {i}")

        # Calculate end index
        j = min(i+bs, nsamples)

        # Prepare inputs and move to device
        inputs = testenc[:,(i * model.seqlen):(j * model.seqlen)].to(device)
        inputs = inputs.reshape(j-i, model.seqlen)

        # Forward pass through the model
        print(inputs.shape)
        lm_logits = model(inputs).logits

        # Shift logits and labels for next token prediction
        shift_logits = lm_logits[:, :-1, :].contiguous()
        shift_labels = inputs[:, 1:]

        # Compute loss
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(shift_logits.reshape(-1, shift_logits.size(-1)), shift_labels.reshape(-1))

        # Calculate negative log likelihood
        neg_log_likelihood = loss.float() * model.seqlen * (j-i)

        # Append to list of negative log likelihoods
        nlls.append(neg_log_likelihood)

    # Compute perplexity
    ppl = torch.exp(torch.stack(nlls).sum() / (nsamples * model.seqlen))

    # Empty CUDA cache to save memory
    torch.cuda.empty_cache()
    print("PPL: ", ppl.item())

    return ppl.item()

def eval_zero_shot(model_name, model, tokenizer, saved_dir, task_list=["boolq","rte","hellaswag","winogrande","arc_challenge","arc_easy","openbookqa"], 
        num_fewshot=0, use_accelerate=False, add_special_tokens=False):
    from lm_eval import tasks, evaluator 
    def pattern_match(patterns, source_list):
        task_names = set()
        for pattern in patterns:
            for matching in fnmatch.filter(source_list, pattern):
                task_names.add(matching)
        return list(task_names)
    
    print("Model passed to evaluation: ", model)
    task_names = pattern_match(task_list, tasks.get_task_dict(["boolq","rte","hellaswag","winogrande","arc_challenge","arc_easy","openbookqa"]).keys())
    model_args = f"pretrained={saved_dir}"
    limit = 100
    if "70b" in model_name or "65b" in model_name:
        limit = 2000
    if use_accelerate:
        model_args = f"pretrained={model_path}, cache_dir={model_path},use_accelerate=True"
    results = evaluator.simple_evaluate(
        model="huggingface",
        model_args=model_args,    ## This is responsible for taking the pruned model
        tasks=task_names,
        num_fewshot=num_fewshot,
        # pretrained_model=model,
        batch_size=None,
        device=None,
        limit=limit,
    )

    return results 

## Pruning Utilities

In [11]:
def find_pruneable_heads_and_indices(
    heads, n_heads, head_size, already_pruned_heads = set()
):
    """
    Finds the heads and their indices taking `already_pruned_heads` into account.

    Args:
        heads (`List[int]`): List of the indices of heads to prune.
        n_heads (`int`): The number of heads in the model.
        head_size (`int`): The size of each head.
        already_pruned_heads (`Set[int]`): A set of already pruned heads.

    Returns:
        `Tuple[Set[int], torch.LongTensor]`: A tuple with the indices of heads to prune taking `already_pruned_heads`
        into account and the indices of rows/columns to keep in the layer weight.
    """
    mask = torch.ones(n_heads, head_size)
    heads = set(heads) - already_pruned_heads  # Convert to set and remove already pruned heads
    for head in heads:
        # Compute how many pruned heads are before the head and move the index accordingly
        head = head - sum(1 if h < head else 0 for h in already_pruned_heads)
        mask[head] = 0
    mask = mask.view(-1).contiguous().eq(1)
    index: torch.LongTensor = torch.arange(len(mask))[mask].long()
    return heads, index



def prune_linear_by_mask(layer: nn.Linear, index: torch.LongTensor, dim: int = 0) -> nn.Linear:
    """
    Masking based Pruning of linear layer, just make entries zeros
    Not change size of the matrices

    Used to remove heads.

    Args:
        layer (`torch.nn.Linear`): The layer to prune.
        index (`torch.LongTensor`): The indices to keep in the layer.
        dim (`int`, *optional*, defaults to 0): The dimension on which to keep the indices.

    Returns:
        `torch.nn.Linear`: The pruned layer as a new layer with `requires_grad=True`.
    """
    index = index.to(layer.weight.device)
    mask = torch.zeros_like(layer.weight)
    if dim == 0:
        mask[index] = 1
    else:
        mask[:, index] = 1

    layer.weight.data *= mask
    layer.weight.requires_grad = True

    if layer.bias is not None:
        bias_mask = torch.zeros_like(layer.bias)
        if dim == 0:
            bias_mask[index] = 1
        else:
            bias_mask = 1
        layer.bias.data *= bias_mask
        layer.bias.requires_grad = True

    return layer

def prune_linear_layer(layer: nn.Linear, index: torch.LongTensor, dim: int = 0) -> nn.Linear:
    """
    Prune a linear layer to keep only entries in index.

    Used to remove heads.

    Args:
        layer (`torch.nn.Linear`): The layer to prune.
        index (`torch.LongTensor`): The indices to keep in the layer.
        dim (`int`, *optional*, defaults to 0): The dimension on which to keep the indices.

    Returns:
        `torch.nn.Linear`: The pruned layer as a new layer with `requires_grad=True`.
    """
    index = index.to(layer.weight.device)
    assert max(index) < layer.weight.size(dim), "Index out of bounds"
    W = layer.weight.index_select(dim, index).clone().detach()
    if layer.bias is not None:
        if dim == 1:
            b = layer.bias.clone().detach()
        else:
            b = layer.bias[index].clone().detach()
    new_size = list(layer.weight.size())
    new_size[dim] = len(index)
    new_layer = nn.Linear(new_size[1], new_size[0], bias=layer.bias is not None).to(layer.weight.device)
    new_layer.weight.requires_grad = False
    new_layer.weight.copy_(W.contiguous())
    new_layer.weight.requires_grad = True
    if layer.bias is not None:
        new_layer.bias.requires_grad = False
        new_layer.bias.copy_(b.contiguous())
        new_layer.bias.requires_grad = True
    return new_layer


def get_heads_to_prune(model, importance_func, threshold):
    import numpy as np
    # Initialize dictionary to hold layers and heads to prune
    heads_to_prune = {}

    # Loop over each layer in the model
    for i, layer in enumerate(model.model.layers):
        # Calculate the importance of the weights of each head in the layer using the provided function
        weights = layer.self_attn.q_proj.weight + layer.self_attn.k_proj.weight + layer.self_attn.v_proj.weight
        head_importances = importance_func(weights.view(layer.self_attn.num_heads, -1), target=torch.zeros_like(weights.view(layer.self_attn.num_heads, -1)), reduction='none').sum(dim=-1)
        head_importances = head_importances.detach().cpu().numpy()
        
        d_prime = int(np.ceil(layer.self_attn.num_heads * threshold))
        print(d_prime)
        sorted_idx = np.argsort(head_importances)
        preserve_idx = sorted_idx[d_prime:]
        prune_indices = sorted_idx[:d_prime]

        # Add the layer and the heads to prune to the dictionary
        heads_to_prune[i] = prune_indices

    return heads_to_prune

def get_heads_to_prune_layer(model, importance_func, threshold, layer_index):
    import numpy as np
    # Initialize dictionary to hold layers and heads to prune
    heads_to_prune = {}

    layers = model.model.layers
    layer = layers[layer_index]
    # Calculate the importance of the weights of each head in the layer using the provided function
    weights = layer.self_attn.q_proj.weight + layer.self_attn.k_proj.weight + layer.self_attn.v_proj.weight
    head_importances = importance_func(weights.view(layer.self_attn.num_heads, -1), target=torch.zeros_like(weights.view(layer.self_attn.num_heads, -1)), reduction='none').sum(dim=-1)
    head_importances = head_importances.detach().cpu().numpy()
    
    d_prime = int(np.ceil(layer.self_attn.num_heads * threshold))
    print(d_prime)
    sorted_idx = np.argsort(head_importances)
    preserve_idx = sorted_idx[d_prime:]
    prune_indices = sorted_idx[:d_prime]

    # Add the layer and the heads to prune to the dictionary

    return prune_indices


def get_inter_indices_to_keep(model, layer, importance_func, threshold):
    # Type : LlamaMLP
    if isinstance(layer, int):
        inter_layer = list(model.modules())[layer]
    else:
        inter_layer = layer
    print(type(inter_layer))
    assert isinstance(inter_layer, LlamaMLP)
    inter_dim = inter_layer.intermediate_size
    d_prime = int(np.ceil(inter_dim *(1 - threshold)))
    print(d_prime)
                  
    
    gate_weights = inter_layer.gate_proj.weight.data.cpu().numpy() 
    up_weights = inter_layer.up_proj.weight.data.cpu().numpy() 
    down_weights= inter_layer.down_proj.weight.data.cpu().numpy()
    down_weights = np.transpose(down_weights)
    
    ## Add importance function here 
    
    importance = torch.from_numpy(np.abs(gate_weights) + np.abs(up_weights) + np.abs(down_weights)).sum(dim=1)
    print(len(importance))
    sorted_idx = np.argsort(-importance)
    preserve_idx = sorted_idx[:d_prime]
    print(preserve_idx)
    prune_idx = sorted_idx[d_prime:]
    return preserve_idx, importance

def find_layers(module, layers=[nn.Linear], name=''):
    """
    Recursively find the layers of a certain type in a module.

    Args:
        module (nn.Module): PyTorch module.
        layers (list): List of layer types to find.
        name (str): Name of the module.

    Returns:
        dict: Dictionary of layers of the given type(s) within the module.
    """
    if type(module) in layers:
        return {name: module}
    res = {}
    for name1, child in module.named_children():
        res.update(find_layers(
            child, layers=layers, name=name + '.' + name1 if name != '' else name1
        ))
    return res

In [12]:
long_input = """Provide a concise summary of the below passage.

Hannah Arendt was one of the seminal political thinkers of the twentieth century. The power and originality of her thinking was evident in works such as The Origins of Totalitarianism, The Human Condition, On Revolution and The Life of the Mind. In these works and in numerous essays she grappled with the most crucial political events of her time, trying to grasp their meaning and historical import, and showing how they affected our categories of moral and political judgment. What was required, in her view, was a new framework that could enable us to come to terms with the twin horrors of the twentieth century, Nazism and Stalinism. She provided such framework in her book on totalitarianism, and went on to develop a new set of philosophical categories that could illuminate the human condition and provide a fresh perspective on the nature of political life.

Although some of her works now belong to the classics of the Western tradition of political thought, she has always remained difficult to classify. Her political philosophy cannot be characterized in terms of the traditional categories of conservatism, liberalism, and socialism. Nor can her thinking be assimilated to the recent revival of communitarian political thought, to be found, for example, in the writings of A. MacIntyre, M. Sandel, C. Taylor and M. Walzer. Her name has been invoked by a number of critics of the liberal tradition, on the grounds that she presented a vision of politics that stood in opposition some key liberal principles. There are many strands of Arendt’s thought that could justify such a claim, in particular, her critique of representative democracy, her stress on civic engagement and political deliberation, her separation of morality from politics, and her praise of the revolutionary tradition. However, it would be a mistake to view Arendt as an anti-liberal thinker. Arendt was in fact a stern defender of constitutionalism and the rule of law, an advocate of fundamental human rights (among which she included not only the right to life, liberty, and freedom of expression, but also the right to action and to opinion), and a critic of all forms of political community based on traditional ties and customs, as well as those based on religious, ethnic, or racial identity.

Arendt’s political thought cannot, in this sense, be identified either with the liberal tradition or with the claims advanced by a number of its critics. Arendt did not conceive of politics as a means for the satisfaction of individual preferences, nor as a way to integrate individuals around a shared conception of the good. Her conception of politics is based instead on the idea of active citizenship, that is, on the value and importance of civic engagement and collective deliberation about all matters affecting the political community. If there is a tradition of thought with which Arendt can be identified, it is the classical tradition of civic republicanism originating in Aristotle and embodied in the writings of Machiavelli, Montesquieu, Jefferson, and Tocqueville. According to this tradition politics finds its authentic expression whenever citizens gather together in a public space to deliberate and decide about matters of collective concern. Political activity is valued not because it may lead to agreement or to a shared conception of the good, but because it enables each citizen to exercise his or her powers of agency, to develop the capacities for judgment and to attain by concerted action some measure of political efficacy."""


# Prompts 

In [13]:
inputs_chat = [
  "Think of some family rules to promote a healthy family relationship",
  "In the series A Song of Ice and Fire, who is the founder of House Karstark?",
  "which weighs more, cold or hot water?",
  "Write a short paragraph about why you should not have both a pet cat and a pet bird.",
  "Is beauty objective or subjective?",
  "What is SVM?",
  "What is the current capital of Japan?",
  "Name 10 colors",
  "How should I invest my money?",
  "What are some ways to improve the value of your home?",
  "What does fasting mean?",
  "What is cloud computing in simple terms?",
  "What is the meaning of life?",
  "What is Linux?",
  "Why do people like gardening?",
  "What makes for a good photograph?"
]




inputs_base = [
    "Gardening provides a sense of accomplishment and tranquility, as you nurture plants and watch them",
    "Family traditions are passed down from generation to generation, creating a sense of continuity and belonging. Whether it's holiday celebrations, special recipes, or annual gatherings, these traditions foster",
    "The evolution of fashion reflects changes in society and culture throughout history. From the elaborate garments of the Victorian era to the minimalist styles of the 21st century, fashion is a form of self-expression that adapts to the times. Iconic designers like Coco Chanel, Alexander McQueen, and Virgil Abloh have challenged conventions and introduced new aesthetics. Fashion is also influenced by technological advancements, such as the development of sustainable materials and the rise of digital fashion shows. As we move towards a more inclusive and diverse industry, fashion will continue to celebrate individuality and innovation, inspiring people to express their unique identities through their clothing choices. In the future",
    
    "Artificial intelligence is transforming the healthcare industry by enabling",
    "Machine learning algorithms are increasingly being used to predict stock market trends. By analyzing historical data and identifying patterns, these models can",
    "The integration of natural language processing with machine learning has opened up new possibilities in human-computer interaction. For instance, chatbots powered by sophisticated NLP algorithms can now understand and respond to user queries with remarkable accuracy. As these technologies continue to evolve, we can expect that",
    
    "Self-driving cars rely heavily on sensor data and advanced algorithms to navigate",
    "One of the biggest challenges in developing AI systems is ensuring they are ethical and unbiased. Researchers are working on creating frameworks that can",
    "In the field of computer vision, convolutional neural networks have revolutionized the way machines perceive visual information. By mimicking the human visual system, these networks can classify images, detect objects, and even perform image segmentation. This has significant applications in areas such as medical imaging, where AI can assist doctors in diagnosing conditions from X-rays and MRIs. As the technology advances, we may soon see AI systems that",
    
    "Reinforcement learning algorithms learn optimal behaviors through",
    "Quantum computing is poised to revolutionize the field of cryptography. With the ability to solve complex problems much faster than classical computers, quantum algorithms could",
    "The rise of big data has necessitated the development of new machine learning techniques capable of handling vast amounts of information. Techniques such as distributed learning and federated learning have emerged to process data across multiple devices without compromising privacy. These methods enable organizations to leverage large datasets for training robust models while ensuring data security. As we look to the future, the continued evolution of these techniques will likely lead to even more powerful and secure AI systems that",
    
    "Transfer learning allows models trained on large datasets to be fine-tuned for specific tasks by",
    "Explainable AI (XAI) is becoming increasingly important as AI systems are deployed in critical areas. By providing insights into how models make decisions, XAI aims to",
    
    "The beauty of a sunset over the ocean is unmatched, as the sky turns shades of orange and pink, and",
    "Traveling to new countries allows you to experience diverse cultures and traditions. Walking through the bustling markets, tasting local delicacies, and interacting with people can provide a deeper understanding of"
]

# Generation Code

## Text Generation 

In [14]:
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
PROMPT_FOR_GENERATION_FORMAT = """
<s>[INST]<<SYS>>
{system_prompt}
<</SYS>>

{instruction}
[/INST]
""".format(
    system_prompt=DEFAULT_SYSTEM_PROMPT,
    instruction="{instruction}"
)



# Define parameters to generate text
def gen_text(prompts, pipeline, tokenizer, use_template=False, **kwargs):
    if use_template:
        full_prompts = [
            PROMPT_FOR_GENERATION_FORMAT.format(instruction=prompt)
            for prompt in prompts
        ]
    else:
        full_prompts = prompts

    if "batch_size" not in kwargs:
        kwargs["batch_size"] = 1
    
    # the default max length is pretty small (20), which would cut the generated output in the middle, so it's necessary to increase the threshold to the complete response
    if "max_new_tokens" not in kwargs:
        kwargs["max_new_tokens"] = 512

    # configure other text generation arguments, see common configurable args here: https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.GenerationConfig
    kwargs.update(
        {
            "pad_token_id": tokenizer.eos_token_id,  # Hugging Face sets pad_token_id to eos_token_id by default; setting here to not see redundant message
            "eos_token_id": tokenizer.eos_token_id,
        }
    )

    outputs = pipeline(full_prompts, **kwargs)
    outputs = [out[0]["generated_text"] for out in outputs]

    return outputs


# results = gen_text(["What is a large language model?"])
# print(results[0])


## Throughput

In [15]:
def get_num_tokens(text):
    inputs = tokenizer(text, return_tensors="pt").input_ids.to("cuda")
    return inputs.shape[1]

In [16]:
def get_gen_text_throughput(prompt, pipeline, tokenizer, use_template=True, **kwargs):
    """
    Return tuple ( number of tokens / sec, num tokens, output ) of the generated tokens
    """
    if use_template:
        full_prompt = PROMPT_FOR_GENERATION_FORMAT.format(instruction=prompt)
    else:
        full_prompt = prompt

    if "max_new_tokens" not in kwargs:
        kwargs["max_new_tokens"] = 512

    kwargs.update(
        {
            "do_sample": True,
            "pad_token_id": tokenizer.eos_token_id,
            "eos_token_id": tokenizer.eos_token_id,
            "return_tensors": True,  # make the pipeline return token ids instead of decoded text to get the number of generated tokens
        }
    )

    num_input_tokens = get_num_tokens(full_prompt)

    # measure the time it takes for text generation
    start = time.time()
    outputs = pipeline(full_prompt, **kwargs)
    duration = time.time() - start

    # get the number of generated tokens
    n_tokens = len(outputs[0]["generated_token_ids"])

    # show the generated text in logging
    result = tokenizer.batch_decode(
        outputs[0]["generated_token_ids"][num_input_tokens:], skip_special_tokens=True
    )
    result = " ".join(result)

    return (n_tokens / duration, n_tokens, result)

# Experiment Runner

In [17]:
tokenizer = AutoTokenizer.from_pretrained(hf_llama_path, padding_side="left", padding=True, truncation=True)
pipeline = transformers.pipeline(
    "text-generation",
    model=hf_llama_path,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
#     revision=revision,
    return_full_text=False
)

# Required tokenizer setting for batch inference
pipeline.tokenizer.pad_token_id = tokenizer.eos_token_id

model = pipeline.model
model.seqlen = model.config.max_position_embeddings 

2024-06-11 10:04:18.805070: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-11 10:04:18.857357: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-11 10:04:19.912321: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
model.num_parameters()

6738415616

In [19]:
import re

def clean_output(output_text):
    # Remove repeated sentences
    unique_sentences = set()
    cleaned_output = []
    
    for sentence in output_text.split('.'):
        sentence = sentence.strip()
        if sentence and sentence not in unique_sentences:
            cleaned_output.append(sentence)
            unique_sentences.add(sentence)
    
    # Remove garbage symbols
#     cleaned_output = [re.sub(r'[^\w\s]', '', sentence) for sentence in cleaned_output]
    
    # Exclude the last sentence if it does not end with a period
    if output_text[-1] != '.':
        cleaned_output = cleaned_output[:-1]
    
    return ". ".join(cleaned_output) + "."

import json
import os
from transformers.models.llama.modeling_llama import LlamaAttention, LlamaDecoderLayer, LlamaMLP, LlamaConfig, LlamaDecoderLayer


class ExperimentRunner:
    def __init__(self, pipeline, tokenizer, pruning_dict, 
                 save_dir=None, output_dir=None, ignored_layers=None, 
                 use_template=False):
        self.pipeline = pipeline
        self.model = self.pipeline.model
        self.tokenizer = tokenizer
        self.pruning_dict = pruning_dict
        self.save_dir = save_dir
        self.output_dir = output_dir
        self.ignored_layers = ignored_layers
        
        self.use_template = use_template
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)
        
    
    def get_head_inter_pruning_dict(self):
        ## load pruning dict
        if isinstance(self.pruning_dict,str):
            pruning_dict_path = "../pruning_dicts/"  # replace with your directory
            file_path = os.path.join(pruning_dict_path, self.pruning_dict)

            with open(file_path, "r") as f:
                data = json.load(f)
            self.pruning_dict = data
        else:
            if isinstance(self.pruning_dict, dict):
                self.pruning_dict = pruning_dict
            else:
                raise ValueError("Pruning dict not valid")
        for i, ind in self.pruning_dict.items():
            print(i, "==> ",len(ind))
    def prune_model(self, real):
        modules = list(self.model.modules())
        for idx, index in self.pruning_dict.items():
            idx = int(idx)
#             print("Pruning idx: {}, Type: {}".format(idx, type(modules[idx])))
            op = modules[idx]
            if isinstance(op, LlamaAttention):
                if real:
                    op.q_proj = prune_linear_layer(op.q_proj, torch.LongTensor(index)).half()
                    op.k_proj = prune_linear_layer(op.k_proj, torch.LongTensor(index)).half()
                    op.v_proj = prune_linear_layer(op.v_proj, torch.LongTensor(index)).half()
                    op.o_proj = prune_linear_layer(op.o_proj, torch.LongTensor(index), dim=1).half()
                    
                    op.num_heads = len(index)//op.head_dim
                    op.num_key_value_heads = op.num_heads
                    op.hidden_size = len(index)
                else:
                    op.q_proj = prune_linear_by_mask(op.q_proj, torch.LongTensor(index)).half()
                    op.k_proj = prune_linear_by_mask(op.k_proj, torch.LongTensor(index)).half()
                    op.v_proj = prune_linear_by_mask(op.v_proj, torch.LongTensor(index)).half()
                    op.o_proj = prune_linear_by_mask(op.o_proj, torch.LongTensor(index), dim=1).half()
            elif isinstance(op, LlamaMLP):
                if real:
                    op.gate_proj = prune_linear_layer(op.gate_proj, torch.LongTensor(index)).half()
                    op.up_proj = prune_linear_layer(op.up_proj, torch.LongTensor(index)).half()
                    op.down_proj = prune_linear_layer(op.down_proj, torch.LongTensor(index), dim=1).half()
                    
                    op.intermediate_size = len(index)
                else:
                    op.gate_proj = prune_linear_by_mask(op.gate_proj, torch.LongTensor(index)).half()
                    op.up_proj = prune_linear_by_mask(op.up_proj, torch.LongTensor(index)).half()
                    op.down_proj = prune_linear_by_mask(op.down_proj, torch.LongTensor(index), dim=1).half()
                    
            else: 
                raise ValueError("Got invalid Module")
            print("Model size after pruning: ", self.get_model_size())
        
    def prune_and_save_checkpoint(self, real=False):
        # Prune the model
        print("Pruning model")
        self.prune_model(real)
        print("Saving model after pruning to checkpoint dir")
        # Save the model checkpoint
        self.model.save_pretrained(self.save_dir)
        self.tokenizer.save_pretrained(self.save_dir)
        print("Model saved")
        
    def save_checkpoint(self):
        self.model.save_pretrained(self.save_dir)
        self.tokenizer.save_pretrained(self.save_dir)
        print("Model saved")
        
    def remove_checkpoint(self):
        import os
        import glob

        folder_path = self.save_dir # replace with your folder path
        extension = '*safetensors'  # replace with your file extension

        files = glob.glob(os.path.join(folder_path, extension))

        for file in files:
            os.remove(file)
        
    def get_model_size(self):
        return self.model.get_memory_footprint()/10e8
    
    def evaluate_ppl(self):
        # Evaluate the model on wikitext and zero shot
        wikitext_results = eval_ppl(self.pipeline.model, self.tokenizer)
        # Dump the results to json files
        print("Perplexity on wikitext2: ", wikitext_results)
        with open(os.path.join(self.output_dir, 'wikitext_results.json'), 'w') as f:
            json.dump(wikitext_results, f, indent=4)
        
    def evaluate_zero_shot(self):
        task_list = ["boolq", "rte","hellaswag","winogrande", "arc_easy","arc_challenge", "openbookqa"]
        num_shot = 0
        print("Loading checkpoint from {}".format(self.save_dir))
        m = "Llama-2-7b-hf"
        accelerate = False
        results = eval_zero_shot(m, self.model, self.tokenizer, self.save_dir, task_list, num_shot, accelerate)

        with open(os.path.join(self.output_dir, 'zero_shot_results.json'), 'w') as f:
            json.dump(results['results'], f, indent=4)
            
        from prettytable import PrettyTable

        # Create a PrettyTable object
        table = PrettyTable()
        # Add columns
        table.field_names = ["Alias", "Acc, None", "Acc StdErr, None", "Acc Norm, None", "Acc Norm StdErr, None"]
        # Add rows
        for key, value in results['results'].items():
            row = [value.get('alias'), value.get('acc,none'), value.get('acc_stderr,none'), value.get('acc_norm,none'), value.get('acc_norm_stderr,none')]
            table.add_row(row)

        print(table)
    
    def retrain(self, train_loader, epochs):
        pass
                
    def get_throughput(self):
        throughput, n_tokens, result = get_gen_text_throughput("What is ML?",self.pipeline, self.tokenizer, 
                                                               max_new_tokens=512, use_template=self.use_template)

        print(f"Short Context: {throughput} tokens/sec, {n_tokens} tokens (including full prompt)")

        # COMMAND ----------

        # When the context is long or the generated text is long, it takes longer to generate each token in average
        throughput, n_tokens, result = get_gen_text_throughput(long_input, self.pipeline, self.tokenizer,
                                                               max_new_tokens=512, use_template=self.use_template)

        print(f"Long Context: {throughput} tokens/sec, {n_tokens} tokens (including full prompt)")
        # print(result)

        t = []
        data = inputs_chat if self.use_template else inputs_base 
        for inp in data:
            throughput, n_tokens, result = get_gen_text_throughput(inp, self.pipeline, 
                                                                   self.tokenizer, max_new_tokens=512, 
                                                                   use_template=self.use_template)
            t.append(throughput)
        print(f"Average:  {sum(t)/len(t)} tokens/sec")

    def generate_from_prompts(self):
        data = inputs_chat if self.use_template else inputs_base 
 
        with open(os.path.join(self.output_dir,'generation_using_prompts.txt'), 'w') as f:
            for inp in data: 
                results = gen_text([inp], self.pipeline, self.tokenizer, temperature=0.6, use_template=self.use_template, batch_size=1)

                print("***" * 50)
                f.write("***" * 50 + '\n')
                print("Original: ")
                f.write("Original: " + '\n')
                print("Prompt:  ", end="")
                f.write("Prompt:  ")
                print("\033[1m {} \033[0m".format(inp), end="")  # \033[1m and \033[0m are the ANSI escape sequences for start and end of bold respectively
                f.write("\033[1m {} \033[0m".format(inp))
                print("\033[91m{}\033[0m".format(results[0]))
                f.write("\033[91m{}\033[0m".format(results[0]) + '\n')

                print("---" * 50)
                f.write("---" * 50 + '\n')
                print("Cleaned: ")
                f.write("Cleaned: " + '\n')
                print("Prompt:  ", end="")
                f.write("Prompt:  ")
                print("\033[1m {} \033[0m".format(inp), end="")  # \033[1m and \033[0m are the ANSI escape sequences for start and end of bold respectively
                f.write("\033[1m {} \033[0m".format(inp))
                print("\033[91m{}\033[0m".format(clean_output(results[0])))
                f.write("\033[91m{}\033[0m".format(clean_output(results[0])) + '\n')

                print('\n')
                f.write('\n')

    def get_eval(self):
        ## Eval on wikitext 
        self.evaluate_ppl()
        ## Eval zero shot 
        self.evaluate_zero_shot()
        ## Get generation on prompts
        self.generate_from_prompts()
        
    def run(self, eval_orig_model=False):
        print("*" * 15, "  Running Experiment  ", "*" * 15)
        self.get_head_inter_pruning_dict()
        if eval_orig_model:
            get_eval()
            
        print("*" * 15, "  Pruning Model  ", "*" * 15)
        self.prune_and_save_checkpoint(real=False)
        
        print("*" * 15, "  Model Pruned Successfully  ", "*" * 15)
        print("Model Size after Pruning: ", self.get_model_size())
        self.get_eval()
        
        self.prune_and_save_checkpoint(real=True)
        print("Real Pruned Model")
        print(self.model)
        
        print("Real Pruned Model Size")
        print(self.get_model_size())
        
        self.get_throughput()
        self.remove_checkpoint()
        print("*" * 15, "  Experiment completed successfully Successfully  ", "*" * 15)
        
            

# Wanda Experiments

## Pr=0.8, lb=0.7, lb=1.0

In [24]:
pruning_dict = "pruning_dict_1.0_0.7_1.0_chat_2640.json"
save_dir = "/data/home/milinbhade/Milin/AMC/bertamc_v4/llama_checkpoints/"

p = pruning_dict.split("_")[-1].split(".")[-2]
output_dir = "/data/home/milinbhade/Milin/AMC/bertamc_v4/llama_checkpoints//" + p

In [25]:
experiment = ExperimentRunner(pipeline, tokenizer, pruning_dict, 
                 save_dir=save_dir, output_dir=output_dir, ignored_layers=None, 
                 use_template=use_template)

In [26]:
experiment.run()

***************   Running Experiment   ***************
61 ==>  4096
67 ==>  10945
75 ==>  4096
81 ==>  10953
89 ==>  4096
95 ==>  10965
103 ==>  4096
109 ==>  10830
117 ==>  4096
123 ==>  10774
131 ==>  4096
137 ==>  10970
145 ==>  4096
151 ==>  10936
159 ==>  4096
165 ==>  10997
173 ==>  4096
179 ==>  10930
187 ==>  4096
193 ==>  9594
201 ==>  2816
207 ==>  7706
215 ==>  2816
221 ==>  7706
229 ==>  2816
235 ==>  7706
243 ==>  2816
249 ==>  7706
257 ==>  2816
263 ==>  7706
271 ==>  2816
277 ==>  7706
285 ==>  2816
291 ==>  7706
299 ==>  2816
305 ==>  7731
313 ==>  2816
319 ==>  7800
327 ==>  2816
333 ==>  7770
341 ==>  2816
347 ==>  7783
355 ==>  2816
361 ==>  7777
369 ==>  2816
375 ==>  7780
383 ==>  2816
389 ==>  7778
397 ==>  2816
403 ==>  7780
411 ==>  2816
417 ==>  7778
425 ==>  2816
431 ==>  7780
***************   Pruning Model   ***************
Pruning model
Model size after pruning:  13.476839424
Model size after pruning:  13.476839424
Model size after pruning:  13.476839424
Mo

WikiText Validation:   0%|                                                                                 | 0/25 [00:00<?, ?it/s]

torch.Size([1, 2048])


WikiText Validation:   8%|█████▊                                                                   | 2/25 [00:01<00:11,  2.01it/s]

torch.Size([1, 2048])


...

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


******************************************************************************************************************************************************
Original: 
Prompt:   Reinforcement learning algorithms learn optimal behaviors through  the process of trial and error.
The most important thing to remember is that the term “learning” is a bit of a misnomer.
The term “learning” is a bit of a misnomer.
The term “learning” is a bit of a misnomer. The term “learning” is a bit of a misnomer. The term “learning” is a bit of a misnomer. The term “learning” is a bit of a misnomer. The term “learning” is a bit of a misnomer. The term “learning” is a bit of a misnomer. The term “learning” is a bit of a misnomer. The term “learning” is a bit of a misnomer. The term “learning” is a bit of a misnomer. The term “learning” is a bit of a misnomer. The term “learning” is a bit of a misnomer. The term “learning” is a bit of a misnomer. The term “learning” is a bit of a misnomer. The term “learning” is a

******************************************************************************************************************************************************
Original: 
Prompt:   Transfer learning allows models trained on large datasets to be fine-tuned for specific tasks by  using the model’s pre-trained parameters.

\subsection{Transferring knowledge from the source domain to the target}

\subsubsection{Knowledge distillation}

Knowledge distillation (KD) is a technique that allows the transfer of knowledge from a large, well-trained model to a smaller, more efficient model. The goal of KD is to reduce the size of the model by using the parameters of the large model to train the smaller model.

\subsubsection{Supervised learning}

Supervised learning is a technique that allows the transfer of knowledge from the source to the target. The goal of supervised learning is to use the parameters of the large model to train the smaller model.

\subsubsection{Unsupervised learning}

Unsupervised lea

******************************************************************************************************************************************************
Original: 
Prompt:   The beauty of a sunset over the ocean is unmatched, as the sky turns shades of orange and pink, and  the sun disappears.
The best time to see the sunset is between 12:00 and 13:00.
The best time to see the sunset is between 12:00 and 13:00. The sun sets at 12:00 and 13:00. The sun sets at 12:00 and 13:00. The sun sets at 12:00 and 13:00. The sun sets at 12:00 and 13:00. The sun sets at 12:00 and 13:00. The sun sets at 12:00 and 13:00. The sun sets at 12:00 and 13:00. The sun sets at 12:00 and 13:00. The sun sets at 12:00 and 13:00. The sun sets at 12:00 and 13:00. The sun sets at 12:00 and 13:00. The sun sets at 12:00 and 13:00. The sun sets at 12:00 and 13:00. The sun sets at 12:00 and 13:00. The sun sets at 12:00 and 13:00. The sun sets at 12:00 and 13:00. The sun sets at 12:00 and 13:00. The sun sets at 12:00 and 

Short Context: 18.720634692112867 tokens/sec, 516 tokens (including full prompt)
Long Context: 47.54139760489992 tokens/sec, 1288 tokens (including full prompt)
Average:  24.983545910972545 tokens/sec
***************   Experiment completed successfully Successfully   ***************


In [27]:
experiment.model.num_parameters()

5673828352

## Pr=0.7, lb=0.6, lb=1.0

In [24]:
pruning_dict = "pruning_dict_1.0_0.6_1.0_chat_2641.json"
save_dir = "/data/home/milinbhade/Milin/AMC/bertamc_v4/llama_checkpoints/"

p = pruning_dict.split("_")[-1].split(".")[-2]
output_dir = "/data/home/milinbhade/Milin/AMC/bertamc_v4/llama_checkpoints//" + p

In [25]:
experiment = ExperimentRunner(pipeline, tokenizer, pruning_dict, 
                 save_dir=save_dir, output_dir=output_dir, ignored_layers=None, 
                 use_template=use_template)

In [26]:
experiment.run()

***************   Running Experiment   ***************
61 ==>  3968
67 ==>  10990
75 ==>  4096
81 ==>  10864
89 ==>  4096
95 ==>  10953
103 ==>  4096
109 ==>  10921
117 ==>  3968
123 ==>  10946
131 ==>  4096
137 ==>  10589
145 ==>  4096
151 ==>  10971
159 ==>  3456
165 ==>  7790
173 ==>  2432
179 ==>  6605
187 ==>  2432
193 ==>  6605
201 ==>  2432
207 ==>  6605
215 ==>  2432
221 ==>  6605
229 ==>  2432
235 ==>  6605
243 ==>  2432
249 ==>  6605
257 ==>  2432
263 ==>  6605
271 ==>  2432
277 ==>  6605
285 ==>  2432
291 ==>  6605
299 ==>  2432
305 ==>  6605
313 ==>  2432
319 ==>  6605
327 ==>  2432
333 ==>  6605
341 ==>  2432
347 ==>  6606
355 ==>  2432
361 ==>  6666
369 ==>  2432
375 ==>  6625
383 ==>  2432
389 ==>  6652
397 ==>  2432
403 ==>  6635
411 ==>  2432
417 ==>  6646
425 ==>  2432
431 ==>  6638
***************   Pruning Model   ***************
Pruning model
Model size after pruning:  13.476839424
Model size after pruning:  13.476839424
Model size after pruning:  13.476839424
Mode

WikiText Validation:   0%|                                                                                 | 0/25 [00:00<?, ?it/s]

torch.Size([1, 2048])


WikiText Validation:   8%|█████▊                                                                   | 2/25 [00:01<00:11,  1.93it/s]

torch.Size([1, 2048])


...

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


******************************************************************************************************************************************************
Original: 
Prompt:   Reinforcement learning algorithms learn optimal behaviors through  the process of rewards and punishment.
Their goal is to predict the future.
Their goal is to predict the future. They are not designed to be used in the same way.
Their goal is to predict the future. They are not designed to be used in the same way. They are not designed to be used in the same way.
Their goal is to predict the future. They are not designed to be used in the same way. They are not designed to be used in the same way. They are not designed to be used in the same way.
Their goal is to predict the future. They are not designed to be used in the same way. They are not designed to be used in the same way. They are not designed to be used in the same way. They are not designed to be used in the same way. They are not designed to be used in t

******************************************************************************************************************************************************
Original: 
Prompt:   Transfer learning allows models trained on large datasets to be fine-tuned for specific tasks by  using a few examples.

\begin{figure}[t]
\centering
\includegraphics[width=0{.9\textwidth}]{figures/overview_v2.png}
\caption{Overview of the proposed method.}
\label{fig:overview}
\end{figure

\section{Proposed method}

\subsection{Motivation}

\subsubsection{Motivation 1: The importance of the dataset}

The first motivation is that the dataset is the most important factor in the field of computer vision.

\subsubsection{Motivation 2: The importance of the dataset}

The second motivation is that the dataset is the most important factor in the field of computer vision.

\subsubsection{M

The third motivation is that the dataset is the most important factor in the field of computer vision.

\end{document}
----------------

Model size after pruning:  13.47264512
Model size after pruning:  13.472202752
Model size after pruning:  13.472202752
Model size after pruning:  13.468663808
Model size after pruning:  13.468663808
Model size after pruning:  13.467312128
Model size after pruning:  13.467312128
Model size after pruning:  13.465174016
Model size after pruning:  13.460979712
Model size after pruning:  13.459456
Model size after pruning:  13.459456
Model size after pruning:  13.449158656
Model size after pruning:  13.449158656
Model size after pruning:  13.448249344
Model size after pruning:  13.427277824
Model size after pruning:  13.348192256
Model size after pruning:  13.293666304
Model size after pruning:  13.185458176
Model size after pruning:  13.130932224
Model size after pruning:  13.022724096
Model size after pruning:  12.968198144
Model size after pruning:  12.859990016
Model size after pruning:  12.805464064
Model size after pruning:  12.697255936
Model size after pruning:  12.642729984
Model s

Short Context: 19.266940263982573 tokens/sec, 362 tokens (including full prompt)
Long Context: 50.05337300840934 tokens/sec, 1288 tokens (including full prompt)
Average:  45.906952071512855 tokens/sec
***************   Experiment completed successfully Successfully   ***************


In [27]:
experiment.model.num_parameters()

5130981376

## Pr=0.9, lb=0.8, lb=1.0

In [24]:
pruning_dict = "pruning_dict_1.0_0.8_1.0_chat_2639.json"
save_dir = "/data/home/milinbhade/Milin/AMC/bertamc_v4/llama_checkpoints/"

p = pruning_dict.split("_")[-1].split(".")[-2]
output_dir = "/data/home/milinbhade/Milin/AMC/bertamc_v4/llama_checkpoints//" + p

In [25]:
experiment = ExperimentRunner(pipeline, tokenizer, pruning_dict, 
                 save_dir=save_dir, output_dir=output_dir, ignored_layers=None, 
                 use_template=use_template)

In [26]:
experiment.run()

***************   Running Experiment   ***************
61 ==>  3328
67 ==>  8806
75 ==>  3328
81 ==>  8806
89 ==>  3328
95 ==>  10973
103 ==>  3328
109 ==>  8806
117 ==>  3328
123 ==>  8806
131 ==>  3328
137 ==>  9539
145 ==>  3328
151 ==>  10370
159 ==>  3328
165 ==>  8806
173 ==>  3328
179 ==>  9644
187 ==>  3584
193 ==>  10059
201 ==>  3328
207 ==>  8815
215 ==>  4096
221 ==>  8806
229 ==>  3328
235 ==>  8806
243 ==>  3328
249 ==>  8806
257 ==>  3328
263 ==>  8806
271 ==>  3328
277 ==>  8806
285 ==>  3840
291 ==>  8806
299 ==>  3328
305 ==>  8806
313 ==>  3328
319 ==>  9285
327 ==>  3328
333 ==>  8806
341 ==>  3328
347 ==>  8806
355 ==>  3584
361 ==>  9692
369 ==>  3328
375 ==>  8806
383 ==>  3328
389 ==>  8806
397 ==>  3328
403 ==>  8806
411 ==>  3840
417 ==>  8806
425 ==>  3328
431 ==>  10968
***************   Pruning Model   ***************
Pruning model
Model size after pruning:  13.476839424
Model size after pruning:  13.476839424
Model size after pruning:  13.476839424
Model s

WikiText Validation:   0%|                                                                                 | 0/25 [00:00<?, ?it/s]

torch.Size([1, 2048])


WikiText Validation:   8%|█████▊                                                                   | 2/25 [00:01<00:12,  1.87it/s]

torch.Size([1, 2048])


...

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


******************************************************************************************************************************************************
Original: 
Prompt:   Reinforcement learning algorithms learn optimal behaviors through  the process of trial and error.
The most common application of reinforcement learning is in the field of robotics.
Reinforcement learning algorithms are used to teach robots how to perform tasks such as walking, walking, and walking.
The most common application of reinforcement learning is in the field of robotics.
Reinforcement learning algorithms are used to teach robots how to perform tasks such as walking, walking, and walking.
The most common application of reinforcement learning is in the field of robotics.
Reinforcement learning algorithms are used to teach robots how to perform tasks such as walking, walking, and walking.
The most common application of reinforcement learning is in the field of robotics.
Reinforcement learning algorithms are us

******************************************************************************************************************************************************
Original: 
Prompt:   Transfer learning allows models trained on large datasets to be fine-tuned for specific tasks by  simply changing the input and output variables.
The most common use of transfer learning is to train a model on a large dataset and then to fine-tune it for specific tasks. This can be done by simply changing the input and output variables.
For example, if you want to train a model on a large dataset and then to fine-tune it for specific tasks, you can do so by simply changing the input and output variables. This can be done by simply changing the input and output variables.
This can be done by simply changing the input and output variables. This can be done by simply changing the input and output variables. This can be done by simply changing the input and output variables. This can be done by simply changing the input 

******************************************************************************************************************************************************
Original: 
Prompt:   Traveling to new countries allows you to experience diverse cultures and traditions. Walking through the bustling markets, tasting local delicacies, and interacting with people can provide a deeper understanding of  the world around you.
Traveling to new countries allows you to experience different cultures and traditions.
Traveling to new countries allows you to experience different cultures and traditions.
Traveling to new countries allows you to experience different cultures and traditions.
Traveling to new countries allows you to experience different cultures and traditions.
Traveling to new countries allows you to experience different cultures and traditions.
Traveling to new countries allows you to experience different cultures and traditions.
Traveling to new countries allows you to experience different cultur

Short Context: 19.93542221243748 tokens/sec, 172 tokens (including full prompt)
Long Context: 48.06337878481866 tokens/sec, 1288 tokens (including full prompt)
Average:  22.81481191153255 tokens/sec
***************   Experiment completed successfully Successfully   ***************


In [27]:
experiment.model.num_parameters()

5829853184

## Pr=0.6, lb=0.5, lb=1.0

In [24]:
pruning_dict = "pruning_dict_1.0_0.5_1.0_chat_2642.json"
save_dir = "/data/home/milinbhade/Milin/AMC/bertamc_v4/llama_checkpoints/"

p = pruning_dict.split("_")[-1].split(".")[-2]
output_dir = "/data/home/milinbhade/Milin/AMC/bertamc_v4/llama_checkpoints//" + p

In [25]:
experiment = ExperimentRunner(pipeline, tokenizer, pruning_dict, 
                 save_dir=save_dir, output_dir=output_dir, ignored_layers=None, 
                 use_template=use_template)

In [26]:
experiment.run()

***************   Running Experiment   ***************
61 ==>  3968
67 ==>  10916
75 ==>  3968
81 ==>  10473
89 ==>  4096
95 ==>  10926
103 ==>  3968
109 ==>  10865
117 ==>  4096
123 ==>  10718
131 ==>  4096
137 ==>  10632
145 ==>  2048
151 ==>  5504
159 ==>  2048
165 ==>  5504
173 ==>  2048
179 ==>  5504
187 ==>  2048
193 ==>  5504
201 ==>  2048
207 ==>  5504
215 ==>  2048
221 ==>  5504
229 ==>  2048
235 ==>  5504
243 ==>  2048
249 ==>  5504
257 ==>  2048
263 ==>  5504
271 ==>  2048
277 ==>  5504
285 ==>  2048
291 ==>  5504
299 ==>  2048
305 ==>  5504
313 ==>  2048
319 ==>  5504
327 ==>  2048
333 ==>  5504
341 ==>  2048
347 ==>  5504
355 ==>  2048
361 ==>  5504
369 ==>  2048
375 ==>  5504
383 ==>  2048
389 ==>  5504
397 ==>  2048
403 ==>  5504
411 ==>  2048
417 ==>  5504
425 ==>  2048
431 ==>  5504
***************   Pruning Model   ***************
Pruning model
Model size after pruning:  13.476839424
Model size after pruning:  13.476839424
Model size after pruning:  13.476839424
Model

WikiText Validation:   0%|                                                    | 0/25 [00:00<?, ?it/s]

torch.Size([1, 2048])


WikiText Validation:   8%|███▌                                        | 2/25 [00:01<00:11,  1.97it/s]

torch.Size([1, 2048])


...

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


******************************************************************************************************************************************************
Original: 
Prompt:   Reinforcement learning algorithms learn optimal behaviors through  the process of selecting the best possible action in a given situation.
Their main goal is to provide a mechanism for learning the best possible actions to take in a given situation.
Their main goal is to provide a method for selecting the best possible actions to take in a.
Their main goal is to provide a.
Their main goal is to provide a. The.
Their main goal is to provide a. The.
Their main goal is to provide a. The. The.
Their main goal is to provide a. The. The. The.
Their main goal is to provide a. The. The. The. The.
Their main goal is to provide a. The. The. The. The. The.
Their main goal is to provide a. The. The. The. The. The. The.
Their main goal is to provide a. The. The. The. The. The. The. The. The. The. The. The. The. The. The. The. The

******************************************************************************************************************************************************
Original: 
Prompt:   Transfer learning allows models trained on large datasets to be fine-tuned for specific tasks by  using the same model.

\begin{figure}
\centering
\includegraphics[width=\linewidth]{fig_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1
------------------------------------------------------------------------------------------------------------------------------------------------------
Cle

Model size after pruning:  13.47264512
Model size after pruning:  13.470384128
Model size after pruning:  13.466189824
Model size after pruning:  13.453041664
Model size after pruning:  13.453041664
Model size after pruning:  13.451026432
Model size after pruning:  13.446832128
Model size after pruning:  13.44331776
Model size after pruning:  13.44331776
Model size after pruning:  13.43619072
Model size after pruning:  13.43619072
Model size after pruning:  13.426950144
Model size after pruning:  13.35984128
Model size after pruning:  13.224574976
Model size after pruning:  13.157466112
Model size after pruning:  13.022199808
Model size after pruning:  12.955090944
Model size after pruning:  12.81982464
Model size after pruning:  12.752715776
Model size after pruning:  12.617449472
Model size after pruning:  12.550340608
Model size after pruning:  12.415074304
Model size after pruning:  12.34796544
Model size after pruning:  12.212699136
Model size after pruning:  12.145590272
Model si

Short Context: 20.504989174926486 tokens/sec, 516 tokens (including full prompt)
Long Context: 49.769442485619344 tokens/sec, 1288 tokens (including full prompt)
Average:  27.971937052617335 tokens/sec
***************   Experiment completed successfully Successfully   ***************


In [27]:
experiment.model.num_parameters()

4588531712

# Experiments

## Experiment 2396 --> PreserveRatio=0.95, lbound=0.8, rbound=1.0

In [20]:
pruning_dict = "pruning_dict_1.0_0.8_1.0_chat_2396.json"
save_dir = "/data/home/milinbhade/Milin/AMC/bertamc_v4/llama_checkpoints/"

p = pruning_dict.split("_")[-1].split(".")[-2]
output_dir = "/data/home/milinbhade/Milin/AMC/bertamc_v4/llama_checkpoints//" + p

In [21]:
experiment = ExperimentRunner(pipeline, tokenizer, pruning_dict, 
                 save_dir=save_dir, output_dir=output_dir, ignored_layers=None, 
                 use_template=use_template)

In [22]:
experiment.run()

***************   Running Experiment   ***************
61 ==>  4096
67 ==>  11007
75 ==>  4096
81 ==>  11007
89 ==>  4096
95 ==>  11005
103 ==>  4096
109 ==>  11007
117 ==>  4096
123 ==>  11008
131 ==>  4096
137 ==>  11008
145 ==>  4096
151 ==>  11004
159 ==>  4096
165 ==>  11006
173 ==>  4096
179 ==>  11002
187 ==>  4096
193 ==>  11007
201 ==>  4096
207 ==>  11008
215 ==>  4096
221 ==>  11006
229 ==>  4096
235 ==>  11008
243 ==>  4096
249 ==>  11008
257 ==>  4096
263 ==>  11004
271 ==>  4096
277 ==>  11007
285 ==>  4096
291 ==>  11006
299 ==>  4096
305 ==>  11006
313 ==>  4096
319 ==>  9577
327 ==>  3328
333 ==>  8806
341 ==>  3328
347 ==>  8806
355 ==>  3328
361 ==>  8806
369 ==>  3328
375 ==>  8806
383 ==>  3328
389 ==>  8806
***************   Pruning Model   ***************
Pruning model
Model size after pruning:  13.51458816
Model size after pruning:  13.51458816
Model size after pruning:  13.51458816
Model size after pruning:  13.51458816
Model size after pruning:  13.51458816
Mo

WikiText Validation:   0%|                                                                                                                                                                                                                                                                                                                           | 0/25 [00:00<?, ?it/s]

torch.Size([1, 2048])


WikiText Validation:   4%|████████████▎                                                                                                                                                                                                                                                                                                      | 1/25 [00:01<00:30,  1.25s/it]

torch.Size([1, 2048])


...

******************************************************************************************************************************************************
Original: 
Prompt:   Quantum computing is poised to revolutionize the field of cryptography. With the ability to solve complex problems much faster than classical computers, quantum algorithms could  potentially break many of the existing cryptographic protocols.
However, there are also potential benefits to quantum computing for cryptography. For example, quantum-resistant cryptographic protocols could be developed to protect against attacks by quantum computers. Additionally, quantum computing could be used to develop new types of cryptographic algorithms that are more secure than those currently in use.
Overall, the impact of quantum computing on cryptography is still uncertain. However, it is clear that the field of cryptography is likely to undergo significant changes in the coming years as a result of the development of quantum com

******************************************************************************************************************************************************
Original: 
Prompt:   Transfer learning allows models trained on large datasets to be fine-tuned for specific tasks by  only using a small amount of data.

\begin{figure}[t]
\centering
\includegraphics[width=0.9\linewidth]{figures/overview.pdf}
\caption{Overview of the proposed method. We first train a model on a large dataset and then fine-tune it on a small dataset. The fine-tuning is done by using the model's parameters and the small dataset. The fine-tuning is done by using the model's parameters and the small dataset. The fine-tuning is done by using the model's parameters and the small dataset. The fine-tuning is done by using the model's parameters and the small dataset. The fine-tuning is done by using the model's parameters and the small dataset. The fine-tuning is done by using the model's parameters and the small dataset. The f

******************************************************************************************************************************************************
Original: 
Prompt:   Traveling to new countries allows you to experience diverse cultures and traditions. Walking through the bustling markets, tasting local delicacies, and interacting with people can provide a deeper understanding of  the world.
Traveling can also help you develop new skills and knowledge. By learning about different cultures, you can gain a better understanding of the world and how it works. This can help you become more successful in your career and life.
Traveling can also help you develop a better understanding of yourself. By exploring new places, you can gain insight into your own values and beliefs. This can help you become more confident and self-aware.
Traveling can also help you develop a better understanding of the environment. By visiting different places, you can gain insight into how different cultures an

/data/home/milinbhade/miniconda3/envs/huggingface/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Short Context: 10.384607383393115 tokens/sec, 516 tokens (including full prompt)
Long Context: 25.555844198775713 tokens/sec, 1288 tokens (including full prompt)
Average:  14.923618028389734 tokens/sec
***************   Experiment completed successfully Successfully   ***************


In [23]:
experiment.model.num_parameters()

6522257408

In [24]:
del experiment.model
del experiment.tokenizer 
del experiment.pipeline

torch.cuda.empty_cache()


del tokenizer
del model
del pipeline

torch.cuda.empty_cache()

## Experiment 2504 --> PreserveRatio=0.9, lbound=0.8, rbound=1.0

In [19]:
pruning_dict = "pruning_dict_1.0_0.8_1.0_chat_2504.json"
save_dir = "/data/home/milinbhade/Milin/AMC/bertamc_v4/llama_checkpoints/"

p = pruning_dict.split("_")[-1].split(".")[-2]
output_dir = "/data/home/milinbhade/Milin/AMC/bertamc_v4/llama_checkpoints//" + p

In [20]:
experiment = ExperimentRunner(pipeline, tokenizer, pruning_dict, 
                 save_dir=save_dir, output_dir=output_dir, ignored_layers=None, 
                 use_template=False)

In [21]:
experiment.run()

***************   Running Experiment   ***************
61 ==>  3968
67 ==>  9876
75 ==>  3712
81 ==>  8908
89 ==>  4096
95 ==>  10464
103 ==>  3328
109 ==>  9240
117 ==>  3968
123 ==>  10427
131 ==>  3968
137 ==>  10163
145 ==>  3968
151 ==>  10317
159 ==>  3712
165 ==>  10697
173 ==>  3712
179 ==>  10797
187 ==>  3968
193 ==>  10806
201 ==>  3840
207 ==>  10805
215 ==>  3584
221 ==>  9539
229 ==>  4096
235 ==>  10074
243 ==>  3328
249 ==>  10955
257 ==>  3328
263 ==>  10135
271 ==>  4096
277 ==>  9850
285 ==>  3968
291 ==>  10163
299 ==>  3840
305 ==>  10411
313 ==>  4096
319 ==>  10869
327 ==>  3328
333 ==>  9100
341 ==>  3328
347 ==>  8806
355 ==>  3328
361 ==>  8806
369 ==>  3328
375 ==>  8806
383 ==>  3328
389 ==>  8806
***************   Pruning Model   ***************
Pruning model
Model size after pruning:  13.51458816
Model size after pruning:  13.51458816
Model size after pruning:  13.51458816
Model size after pruning:  13.51458816
Model size after pruning:  13.51458816
Model 

WikiText Validation:   0%|                                                                                                                                                                                                                                                                                                                           | 0/25 [00:00<?, ?it/s]

torch.Size([1, 2048])


WikiText Validation:   4%|████████████▎                                                                                                                                                                                                                                                                                                      | 1/25 [00:01<00:25,  1.06s/it]

torch.Size([1, 2048])


...

/data/home/milinbhade/miniconda3/envs/huggingface/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


******************************************************************************************************************************************************
Original: 
Prompt:   Quantum computing is poised to revolutionize the field of cryptography. With the ability to solve complex problems much faster than classical computers, quantum algorithms could  potentially break many of the cryptographic algorithms that are currently in use.





















































































































































































































































































































































































































	











	



	
	
	

	











	


	







	
	
	
	




	
	




	

	
	

	
	





	
	

---------------------------------------------------------------------

******************************************************************************************************************************************************
Original: 
Prompt:   The beauty of a sunset over the ocean is unmatched, as the sky turns shades of orange and pink, and  the sun slowly disapp...

## Gameplay

The game is a 2-to--ad-and-up-to-4-ad-and-up-to-6-ad-and-up-to-8-ad-and-up-to-10-ad-and-up-to-12-ad-and-up-to-14-ad-and-up-to-16-ad-and-up-to-18-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20-ad-and-up-to-20

/data/home/milinbhade/miniconda3/envs/huggingface/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Short Context: 20.326270996495822 tokens/sec, 24 tokens (including full prompt)
Long Context: 41.43768205141343 tokens/sec, 1288 tokens (including full prompt)
Average:  20.555450386164807 tokens/sec
***************   Experiment completed successfully Successfully   ***************


In [23]:
experiment.model.num_parameters()

6277746688

In [ ]:
## 0.95
+---------------+-----------+----------------------+----------------+-----------------------+
|     Alias     | Acc, None |   Acc StdErr, None   | Acc Norm, None | Acc Norm StdErr, None |
+---------------+-----------+----------------------+----------------+-----------------------+
|     boolq     |    0.74   | 0.04408440022768078  |      None      |          None         |
| arc_challenge |    0.42   | 0.04960449637488584  |      0.43      |  0.049756985195624284 |
|   hellaswag   |    0.53   | 0.050161355804659205 |      0.68      |  0.04688261722621504  |
|   openbookqa  |    0.31   | 0.04648231987117316  |      0.44      |  0.04988876515698589  |
|   winogrande  |    0.69   | 0.04648231987117316  |      None      |          None         |
|    arc_easy   |    0.7    | 0.046056618647183814 |      0.76      |  0.04292346959909284  |
|      rte      |    0.72   | 0.045126085985421276 |      None      |          None         |
+---------------+-----------+----------------------+----------------+-----------------------+

In [ ]:
## 0.9
+---------------+-----------+----------------------+----------------+-----------------------+
|     Alias     | Acc, None |   Acc StdErr, None   | Acc Norm, None | Acc Norm StdErr, None |
+---------------+-----------+----------------------+----------------+-----------------------+
|      rte      |    0.58   | 0.04960449637488584  |      None      |          None         |
|   openbookqa  |    0.22   | 0.04163331998932268  |      0.37      |  0.04852365870939098  |
|     boolq     |    0.69   | 0.04648231987117316  |      None      |          None         |
|   hellaswag   |    0.48   | 0.050211673156867795 |      0.64      |  0.04824181513244218  |
|    arc_easy   |    0.59   | 0.04943110704237102  |      0.67      |  0.04725815626252607  |
| arc_challenge |    0.42   | 0.049604496374885836 |      0.42      |  0.04960449637488584  |
|   winogrande  |    0.6    | 0.049236596391733084 |      None      |          None         |
+---------------+-----------+----------------------+----------------+-----------------------+

In [ ]:
## 0.8
+---------------+-----------+----------------------+----------------+-----------------------+
|     Alias     | Acc, None |   Acc StdErr, None   | Acc Norm, None | Acc Norm StdErr, None |
+---------------+-----------+----------------------+----------------+-----------------------+
|   openbookqa  |    0.21   | 0.040936018074033256 |      0.4       |  0.049236596391733084 |
|      rte      |    0.63   | 0.04852365870939099  |      None      |          None         |
| arc_challenge |    0.29   | 0.04560480215720684  |      0.36      |  0.04824181513244218  |
|   winogrande  |    0.69   | 0.04648231987117316  |      None      |          None         |
|     boolq     |    0.67   | 0.04725815626252609  |      None      |          None         |
|    arc_easy   |    0.51   | 0.05024183937956912  |      0.53      |  0.05016135580465919  |
|   hellaswag   |    0.45   | 0.049999999999999996 |      0.65      |   0.0479372485441102  |
+---------------+-----------+----------------------+----------------+-----------------------+

In [ ]:
## 0.7
+---------------+-----------+----------------------+----------------+-----------------------+
|     Alias     | Acc, None |   Acc StdErr, None   | Acc Norm, None | Acc Norm StdErr, None |
+---------------+-----------+----------------------+----------------+-----------------------+
|      rte      |    0.64   | 0.048241815132442176 |      None      |          None         |
| arc_challenge |    0.33   | 0.047258156262526045 |      0.32      |  0.046882617226215034 |
|   winogrande  |    0.66   | 0.04760952285695237  |      None      |          None         |
|    arc_easy   |    0.53   | 0.050161355804659205 |      0.54      |  0.05009082659620332  |
|   hellaswag   |    0.39   | 0.04902071300001974  |      0.56      |  0.04988876515698589  |
|   openbookqa  |    0.28   | 0.04512608598542128  |      0.43      |  0.04975698519562428  |
|     boolq     |    0.71   | 0.04560480215720685  |      None      |          None         |
+---------------+-----------+----------------------+----------------+-----------------------+


In [ ]:
## 0.6
+---------------+-----------+----------------------+----------------+-----------------------+
|     Alias     | Acc, None |   Acc StdErr, None   | Acc Norm, None | Acc Norm StdErr, None |
+---------------+-----------+----------------------+----------------+-----------------------+
|    arc_easy   |    0.41   | 0.049431107042371025 |      0.36      |  0.04824181513244218  |
|   winogrande  |    0.55   |         0.05         |      None      |          None         |
|   hellaswag   |    0.36   | 0.04824181513244218  |      0.49      |  0.05024183937956912  |
|     boolq     |    0.45   |         0.05         |      None      |          None         |
|      rte      |    0.57   | 0.04975698519562428  |      None      |          None         |
|   openbookqa  |    0.14   | 0.03487350880197769  |      0.29      |  0.045604802157206845 |
| arc_challenge |    0.21   | 0.040936018074033256 |      0.31      |  0.04648231987117316  |
+---------------+-----------+----------------------+----------------+-----------------------+


## Experiment 2540 --> PreserveRatio=0.8, lbound=0.7, rbound=1.0

In [19]:
pruning_dict = "pruning_dict_1.0_0.7_1.0_chat_2540.json"
save_dir = "/data/home/milinbhade/Milin/AMC/bertamc_v4/llama_checkpoints/"

p = pruning_dict.split("_")[-1].split(".")[-2]
output_dir = "/data/home/milinbhade/Milin/AMC/bertamc_v4/llama_checkpoints//" + p

In [20]:
experiment = ExperimentRunner(pipeline, tokenizer, pruning_dict, 
                 save_dir=save_dir, output_dir=output_dir, ignored_layers=None, 
                 use_template=False)

In [21]:
experiment.run()

***************   Running Experiment   ***************
61 ==>  4096
67 ==>  11008
75 ==>  4096
81 ==>  11004
89 ==>  4096
95 ==>  11003
103 ==>  4096
109 ==>  11008
117 ==>  4096
123 ==>  11007
131 ==>  4096
137 ==>  11007
145 ==>  4096
151 ==>  10994
159 ==>  4096
165 ==>  11000
173 ==>  4096
179 ==>  8538
187 ==>  2816
193 ==>  7706
201 ==>  2816
207 ==>  7706
215 ==>  2816
221 ==>  7706
229 ==>  2816
235 ==>  7767
243 ==>  2816
249 ==>  7784
257 ==>  2816
263 ==>  7777
271 ==>  2816
277 ==>  7780
285 ==>  2816
291 ==>  7778
299 ==>  2816
305 ==>  7780
313 ==>  2816
319 ==>  7778
327 ==>  2816
333 ==>  7780
341 ==>  2816
347 ==>  7778
355 ==>  2816
361 ==>  7780
369 ==>  2816
375 ==>  7778
383 ==>  2816
389 ==>  7780
***************   Pruning Model   ***************
Pruning model
Model size after pruning:  13.51458816
Model size after pruning:  13.51458816
Model size after pruning:  13.51458816
Model size after pruning:  13.51458816
Model size after pruning:  13.51458816
Model size a

WikiText Validation:   0%|                                                                                                                                                                                                                                                                                                                           | 0/25 [00:00<?, ?it/s]

torch.Size([1, 2048])


WikiText Validation:   4%|████████████▎                                                                                                                                                                                                                                                                                                      | 1/25 [00:00<00:14,  1.64it/s]

torch.Size([1, 2048])


...

******************************************************************************************************************************************************
Original: 
Prompt:   Quantum computing is poised to revolutionize the field of cryptography. With the ability to solve complex problems much faster than classical computers, quantum algorithms could  potentially be used to break the cryptographic codes that protect the digital world from malicious attacks.
The problem is that the technology is still in its early stages and the algorithms are not yet reliable enough to be used in the real world. However, the threat of a quantum cryptography-powered attack is a real one.
The threat of a quantum cryptography-powered attack is a real one.
The threat of a quantum cryptography-powered attack is a real one. The technology is still in its early stages and the algorithms are not yet reliable enough to be used in the real world. However, the threat is a real one.
The threat of a quantum cryptograp

******************************************************************************************************************************************************
Original: 
Prompt:   Transfer learning allows models trained on large datasets to be fine-tuned for specific tasks by  adding a few thousand to a few hundred thousand times of the way data.

\begin{itemize}
\item \textbf{Transfer learning} is a method of learning that allows the use of a model that has been developed to address a problem in a different domain, field, or time in a different field, time, or region of the world.
\end{itemize}

\begin{itemize}
\item \textbf{Machine learning} is a field of study that focuses on the development of software that allows computers to learn, or learn to do, a variety of tasks.
\end{itemize}

\begin{itemize}
\item \textbf{Artificial intelligence} is a field of study that focuses on the development of software that allows computers to learn, or learn to do, a variety of tasks.
\end{itemize}

\begin{

******************************************************************************************************************************************************
Original: 
Prompt:   Traveling to new countries allows you to experience diverse cultures and traditions. Walking through the bustling markets, tasting local delicacies, and interacting with people can provide a deeper understanding of  the world’s history and heritage.
Traveling to other countries allows you to learn about the history of the world and the history of the world’s countries. You can learn about the history of the world and the history of the world’s countries. You can learn about the history of the world and the history of the world’s countries. You can learn about the history of the world and the history of the world’s countries. You can learn about the history of the world and the history of the world’s countries. You can learn about the history of the world and the history of the world’s countries. You can learn about t

/data/home/milinbhade/miniconda3/envs/huggingface/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Short Context: 17.223949437471063 tokens/sec, 516 tokens (including full prompt)
Long Context: 42.66546031921937 tokens/sec, 1288 tokens (including full prompt)
Average:  20.096777656850374 tokens/sec
***************   Experiment completed successfully Successfully   ***************


In [22]:
experiment.model.num_parameters()

5795127296

## Experiment 2496 --> PreserveRatio=0.7, lbound=0.5, rbound=1.0

In [19]:
pruning_dict = "pruning_dict_1.0_0.5_1.0_chat_2496.json"
save_dir = "/data/home/milinbhade/Milin/AMC/bertamc_v4/llama_checkpoints/"

p = pruning_dict.split("_")[-1].split(".")[-2]
output_dir = "/data/home/milinbhade/Milin/AMC/bertamc_v4/llama_checkpoints//" + p

In [20]:
experiment = ExperimentRunner(pipeline, tokenizer, pruning_dict, 
                 save_dir=save_dir, output_dir=output_dir, ignored_layers=None, 
                 use_template=False)

In [21]:
experiment.run()

***************   Running Experiment   ***************
61 ==>  3968
67 ==>  11000
75 ==>  4096
81 ==>  10223
89 ==>  4096
95 ==>  10973
103 ==>  3840
109 ==>  10988
117 ==>  4096
123 ==>  10923
131 ==>  4096
137 ==>  11002
145 ==>  3840
151 ==>  10506
159 ==>  3968
165 ==>  10584
173 ==>  3968
179 ==>  10594
187 ==>  4096
193 ==>  10945
201 ==>  3840
207 ==>  6370
215 ==>  2048
221 ==>  5504
229 ==>  2048
235 ==>  5504
243 ==>  2048
249 ==>  5504
257 ==>  2048
263 ==>  5504
271 ==>  2048
277 ==>  5504
285 ==>  2048
291 ==>  5504
299 ==>  2048
305 ==>  5504
313 ==>  2048
319 ==>  5504
327 ==>  2048
333 ==>  5504
341 ==>  2048
347 ==>  5504
355 ==>  2048
361 ==>  5504
369 ==>  2048
375 ==>  5504
383 ==>  2048
389 ==>  5504
***************   Pruning Model   ***************
Pruning model
Model size after pruning:  13.51458816
Model size after pruning:  13.51458816
Model size after pruning:  13.51458816
Model size after pruning:  13.51458816
Model size after pruning:  13.51458816
Model size

WikiText Validation:   0%|                                                                                                                                                                                                                                                                                                                           | 0/25 [00:00<?, ?it/s]

torch.Size([1, 2048])


WikiText Validation:   4%|████████████▎                                                                                                                                                                                                                                                                                                      | 1/25 [00:00<00:14,  1.63it/s]

torch.Size([1, 2048])


...

******************************************************************************************************************************************************
Original: 
Prompt:   Quantum computing is poised to revolutionize the field of cryptography. With the ability to solve complex problems much faster than classical computers, quantum algorithms could  potentially create a threat to the security of cryptographic systems that are currently in place.
The threat of quantum computing to cryptography is a serious issue that could have a significant impact on the security of the internet. The threat of quantum computing to cryptography is a serious issue that could have a significant impact on the security of the internet. The threat of quantum computing to cryptography is a serious issue that could have a significant impact on the security of the internet. The threat of quantum computing to cryptography is a serious issue that could have a significant impact on the security of the internet. The

******************************************************************************************************************************************************
Original: 
Prompt:   Explainable AI (XAI) is becoming increasingly important as AI systems are deployed in critical areas. By providing insights into how models make decisions, XAI aims to  help users understand and control the power of AI.
The goal of this project is to develop a framework for implementing the concept of transparency in the context of the AI field. The objective is to develop a set of tools that will enable the implementation of the concept of transparency in the context of the AI field.
The objective of the project is to develop a set of tools that will enable the implementation of the concept of transparency in the context of the AI field.
The objective of the project is to develop a set of tools that will enable the implementation of the concept of transparency in the context of the AI field. The objective of the pro

Model size after pruning:  13.510393856
Model size after pruning:  13.510197248
Model size after pruning:  13.510197248
Model size after pruning:  13.490905088
Model size after pruning:  13.490905088
Model size after pruning:  13.490044928
Model size after pruning:  13.48165632
Model size after pruning:  13.4811648
Model size after pruning:  13.4811648
Model size after pruning:  13.47907584
Model size after pruning:  13.47907584
Model size after pruning:  13.478928384
Model size after pruning:  13.470539776
Model size after pruning:  13.458202624
Model size after pruning:  13.45400832
Model size after pruning:  13.443588096
Model size after pruning:  13.439393792
Model size after pruning:  13.429219328
Model size after pruning:  13.429219328
Model size after pruning:  13.42767104
Model size after pruning:  13.419282432
Model size after pruning:  13.305298944
Model size after pruning:  13.23819008
Model size after pruning:  13.102923776
Model size after pruning:  13.035814912
Model size

/data/home/milinbhade/miniconda3/envs/huggingface/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Short Context: 16.46419464722564 tokens/sec, 516 tokens (including full prompt)
Long Context: 40.9696348745398 tokens/sec, 1288 tokens (including full prompt)
Average:  18.287360287072584 tokens/sec
***************   Experiment completed successfully Successfully   ***************


In [22]:
experiment.model.num_parameters()

5318332416

## Experiment 2526 --> PreserveRatio=0.6, lbound=0.5, rbound=1.0

In [20]:
pruning_dict = "pruning_dict_1.0_0.5_1.0_chat_2526.json"
save_dir = "/data/home/milinbhade/Milin/AMC/bertamc_v4/llama_checkpoints/"

p = pruning_dict.split("_")[-1].split(".")[-2]
output_dir = "/data/home/milinbhade/Milin/AMC/bertamc_v4/llama_checkpoints//" + p

In [21]:
experiment = ExperimentRunner(pipeline, tokenizer, pruning_dict, 
                 save_dir=save_dir, output_dir=output_dir, ignored_layers=None, 
                 use_template=False)

In [22]:
experiment.run()

***************   Running Experiment   ***************
61 ==>  3584
67 ==>  5504
75 ==>  3584
81 ==>  5504
89 ==>  3072
95 ==>  8965
103 ==>  2944
109 ==>  5504
117 ==>  3968
123 ==>  6234
131 ==>  3840
137 ==>  5504
145 ==>  3200
151 ==>  5504
159 ==>  3968
165 ==>  5504
173 ==>  3840
179 ==>  5504
187 ==>  3200
193 ==>  5504
201 ==>  3584
207 ==>  5504
215 ==>  2432
221 ==>  6185
229 ==>  3968
235 ==>  5511
243 ==>  3968
249 ==>  5504
257 ==>  3840
263 ==>  5504
271 ==>  3712
277 ==>  5504
285 ==>  3456
291 ==>  5504
299 ==>  3072
305 ==>  5504
313 ==>  2048
319 ==>  5504
327 ==>  2048
333 ==>  5504
341 ==>  2048
347 ==>  5504
355 ==>  2048
361 ==>  5504
369 ==>  2048
375 ==>  5504
383 ==>  2048
389 ==>  5504
***************   Pruning Model   ***************
Pruning model
Model size after pruning:  13.51458816
Model size after pruning:  13.51458816
Model size after pruning:  13.51458816
Model size after pruning:  13.51458816
Model size after pruning:  13.51458816
Model size after pru

WikiText Validation:   0%|                                                                                                                                                                                                                                                                                                                           | 0/25 [00:00<?, ?it/s]

torch.Size([1, 2048])


WikiText Validation:   4%|████████████▎                                                                                                                                                                                                                                                                                                      | 1/25 [00:00<00:13,  1.75it/s]

torch.Size([1, 2048])


...

******************************************************************************************************************************************************
Original: 
Prompt:   Quantum computing is poised to revolutionize the field of cryptography. With the ability to solve complex problems much faster than classical computers, quantum algorithms could  be used to explain the complexity of a cryptographic algorithm. This could be a problem for the algorithm. This is a problem that the algorithm has to be able to solve the problem. This is a problem that the algorithm has to be able to solve the problem. This is a problem that the algorithm has to be able to solve the problem. This is a problem that the algorithm has to be able to solve the problem. This is a problem that the algorithm has to be able to solve the problem. This is a problem that the algorithm has to be able to solve the problem. This is a problem that the algorithm has to be able to solve the problem. This is a problem that t

******************************************************************************************************************************************************
Original: 
Prompt:   The beauty of a sunset over the ocean is unmatched, as the sky turns shades of orange and pink, and  the clouds are the last to disappear.
The sun is a star in the sky, and it is the sun that makes the sky. The sky is a sky, and it is a sky that is a sky. The sky is a sky, and it is a sky, and it is a sky, and it is a sky, and it is a sky, and it is a sky, and it is a sky, and it is a sky, and it is a, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, an

Model size after pruning:  10.7886592
Model size after pruning:  10.653392896
Model size after pruning:  10.586284032
Model size after pruning:  10.451017728
Model size after pruning:  10.383908864
Model size after pruning:  10.24864256
Model size after pruning:  10.181533696
Model size after pruning:  10.046267392
Model size after pruning:  9.979158528
Model size after pruning:  9.843892224
Model size after pruning:  9.77678336
Model size after pruning:  9.641517056
Saving model after pruning to checkpoint dir
Model saved
Real Pruned Model
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-3): 4 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=409

/data/home/milinbhade/miniconda3/envs/huggingface/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Short Context: 17.15873457291694 tokens/sec, 134 tokens (including full prompt)
Long Context: 197.79836299859983 tokens/sec, 846 tokens (including full prompt)
Average:  18.43316377623611 tokens/sec
***************   Experiment completed successfully Successfully   ***************


In [23]:
experiment.model.num_parameters()

4801880064

# Timer Hook

In [20]:
from transformers.models.llama.modeling_llama import LlamaAttention, LlamaDecoderLayer, LlamaMLP, LlamaConfig, LlamaDecoderLayer

class TimerHook:
    def __init__(self):
        self.start_event = torch.cuda.Event(enable_timing=True)
        self.end_event = torch.cuda.Event(enable_timing=True)
        self.total_time = 0
        self.count = 0

    def pre_hook(self, module, input):
#         print("Prehook recorded")
        self.start_event.record()

    def forward_hook(self, module, input, output):
#         print("Post Hook recorded")
        self.end_event.record()
        # Wait for all operations to finish before measuring the time
        torch.cuda.synchronize()
        # elapsed_time gives time in milliseconds. Multiply by 1000 for microseconds.
        self.total_time += self.start_event.elapsed_time(self.end_event) * 1000
        self.count += 1

    def get_average_time(self):
#         print("Total Time: ", self.total_time)
#         print("Count: ", self.count)
        return self.total_time / self.count if self.count > 0 else 0
    def print_params(self):
        return "Total Time: {}, Count: {}".format(self.total_time, self.count)
    
    def reset(self):
        self.total_time = 0
        self.count = 0

In [21]:
from transformers.models.llama.modeling_llama import LlamaAttention, LlamaDecoderLayer, LlamaMLP, LlamaConfig, LlamaDecoderLayer


In [19]:
tokenizer = AutoTokenizer.from_pretrained(hf_llama_path, padding_side="left", padding=True, truncation=True)
pipeline = transformers.pipeline(
    "text-generation",
    model=hf_llama_path,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
#     revision=revision,
    return_full_text=False
)

# Required tokenizer setting for batch inference
pipeline.tokenizer.pad_token_id = tokenizer.eos_token_id

model = pipeline.model
model.seqlen = model.config.max_position_embeddings 

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
2024-05-25 13:15:42.223078: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-25 13:15:43.469643: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/data/home/milinbhade/miniconda3/envs/huggingface/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [20]:
for name, _ in model.named_modules():
    print(name)


model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.self_attn.rotary_emb
model.layers.0.mlp
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.act_fn
model.layers.0.input_layernorm
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.rotary_emb
model.layers.1.mlp
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.act_fn
model.layers.1.input_layernorm
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.

In [21]:
hooks = {name: TimerHook() for name, _ in model.named_modules() if isinstance(_, LlamaDecoderLayer)}
hook_handles = []

In [22]:
pipeline.model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Lin

In [23]:
for name, module in model.named_modules():
    if isinstance(module, LlamaDecoderLayer):
        pre_hook_handle = module.register_forward_pre_hook(hooks[name].pre_hook)
        forward_hook_handle = module.register_forward_hook(hooks[name].forward_hook)
        hook_handles.extend([pre_hook_handle, forward_hook_handle])
        print(hook_handles)

[<torch.utils.hooks.RemovableHandle object at 0x7f8c045ddbe0>, <torch.utils.hooks.RemovableHandle object at 0x7f8c045ddd30>]
[<torch.utils.hooks.RemovableHandle object at 0x7f8c045ddbe0>, <torch.utils.hooks.RemovableHandle object at 0x7f8c045ddd30>, <torch.utils.hooks.RemovableHandle object at 0x7f8c045dd2b0>, <torch.utils.hooks.RemovableHandle object at 0x7f8c045dd3d0>]
[<torch.utils.hooks.RemovableHandle object at 0x7f8c045ddbe0>, <torch.utils.hooks.RemovableHandle object at 0x7f8c045ddd30>, <torch.utils.hooks.RemovableHandle object at 0x7f8c045dd2b0>, <torch.utils.hooks.RemovableHandle object at 0x7f8c045dd3d0>, <torch.utils.hooks.RemovableHandle object at 0x7f8c045ddd60>, <torch.utils.hooks.RemovableHandle object at 0x7f8c045ddd90>]
[<torch.utils.hooks.RemovableHandle object at 0x7f8c045ddbe0>, <torch.utils.hooks.RemovableHandle object at 0x7f8c045ddd30>, <torch.utils.hooks.RemovableHandle object at 0x7f8c045dd2b0>, <torch.utils.hooks.RemovableHandle object at 0x7f8c045dd3d0>, <tor

In [24]:
def print_hooks(model):
    for name, module in model.named_modules():
        if module._forward_hooks or module._forward_pre_hooks or module._backward_hooks:
            print(f'Module: {name}')
            print('Forward Hooks:')
            for hook in module._forward_hooks.values():
                print(hook)
            print('Forward Pre Hooks:')
            for hook in module._forward_pre_hooks.values():
                print(hook)
            print('Backward Hooks:')
            for hook in module._backward_hooks.values():
                print(hook)
print_hooks(model)

Module: model.layers.0
Forward Hooks:
<bound method TimerHook.forward_hook of <__main__.TimerHook object at 0x7f8c1b2f5520>>
Forward Pre Hooks:
<bound method TimerHook.pre_hook of <__main__.TimerHook object at 0x7f8c1b2f5520>>
Backward Hooks:
Module: model.layers.1
Forward Hooks:
<bound method TimerHook.forward_hook of <__main__.TimerHook object at 0x7f8c1b2f53a0>>
Forward Pre Hooks:
<bound method TimerHook.pre_hook of <__main__.TimerHook object at 0x7f8c1b2f53a0>>
Backward Hooks:
Module: model.layers.2
Forward Hooks:
<bound method TimerHook.forward_hook of <__main__.TimerHook object at 0x7f8c1b2f5760>>
Forward Pre Hooks:
<bound method TimerHook.pre_hook of <__main__.TimerHook object at 0x7f8c1b2f5760>>
Backward Hooks:
Module: model.layers.3
Forward Hooks:
<bound method TimerHook.forward_hook of <__main__.TimerHook object at 0x7f8c1b3bf700>>
Forward Pre Hooks:
<bound method TimerHook.pre_hook of <__main__.TimerHook object at 0x7f8c1b3bf700>>
Backward Hooks:
Module: model.layers.4
Forwa

In [25]:
# Create events for the start and end times
start_event = torch.cuda.Event(enable_timing=True)
end_event = torch.cuda.Event(enable_timing=True)

start_event.record()

# Your code here
results = gen_text(["What is a large language model?"], pipeline, tokenizer, temperature=0.5, max_new_tokens=512, use_template=use_template)
print(results[0])

end_event.record()

# Wait for all operations to finish before measuring the time
torch.cuda.synchronize()

# Calculate and print the elapsed time
elapsed_time = start_event.elapsed_time(end_event)  # Time in milliseconds
print(f'Elapsed time: {elapsed_time} ms')

Prehook recorded


/data/home/milinbhade/miniconda3/envs/huggingface/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook r

Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook r

Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook r

...

Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook r

Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook r

Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook r

Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook r

Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook recorded
Post Hook recorded
Prehook r

In [26]:
layers = []
for name, hook in hooks.items():
    avg = hook.get_average_time()
    print(f'Average time for {name}: {avg} milliseconds')
    layers.append(avg)
print(layers)

Total Time:  1592082.4680924416
Count:  512
Average time for model.layers.0: 3109.53607049305 milliseconds
Total Time:  1146662.913262844
Count:  512
Average time for model.layers.1: 2239.5760024664924 milliseconds
Total Time:  1125716.9921398163
Count:  512
Average time for model.layers.2: 2198.6660002730787 milliseconds
Total Time:  1106977.7913689613
Count:  512
Average time for model.layers.3: 2162.0659987675026 milliseconds
Total Time:  1103168.5115098953
Count:  512
Average time for model.layers.4: 2154.6259990427643 milliseconds
Total Time:  1113137.1515393257
Count:  512
Average time for model.layers.5: 2174.0959991002455 milliseconds
Total Time:  1096453.0888795853
Count:  512
Average time for model.layers.6: 2141.50993921794 milliseconds
Total Time:  1086970.878958702
Count:  512
Average time for model.layers.7: 2122.989997966215 milliseconds
Total Time:  1131537.4071598053
Count:  512
Average time for model.layers.8: 2210.0339983589947 milliseconds
Total Time:  1117086.72016

In [42]:
# When you're done, you can remove the hooks like this:
for handle in hook_handles:
    handle.remove()

In [37]:
len(tokenizer.encode(results[0]))

514

In [44]:
print_hooks(model)

# Layer time analysis

In [22]:
from transformers.models.llama.modeling_llama import LlamaAttention, LlamaDecoderLayer, LlamaMLP, LlamaConfig, LlamaDecoderLayer

class TimerHook:
    def __init__(self):
        self.start_event = torch.cuda.Event(enable_timing=True)
        self.end_event = torch.cuda.Event(enable_timing=True)
        self.total_time = 0
        self.count = 0

    def pre_hook(self, module, input):
#         print("Prehook recorded")
        self.start_event.record()

    def forward_hook(self, module, input, output):
#         print("Post Hook recorded")
        self.end_event.record()
        # Wait for all operations to finish before measuring the time
        torch.cuda.synchronize()
        # elapsed_time gives time in milliseconds. Multiply by 1000 for microseconds.
        self.total_time += self.start_event.elapsed_time(self.end_event) * 1000
        self.count += 1

    def get_average_time(self):
#         print("Total Time: ", self.total_time)
#         print("Count: ", self.count)
        return self.total_time / self.count if self.count > 0 else 0
    def print_params(self):
        return "Total Time: {}, Count: {}".format(self.total_time, self.count)
    
    def reset(self):
        self.total_time = 0
        self.count = 0

In [23]:
class LayerAnalysis:
    def __init__(self, pipeline, tokenizer, pruning_dict, 
                 inputs=None, save_dir=None, output_dir=None, 
                 ignore_layers=None, use_template=False):
        self.pipeline = pipeline
        self.model = pipeline.model
        self.tokenizer = tokenizer
        self.pruning_dict = pruning_dict
        
        ### Hooks
        self.hooks = None
        self.hook_handles = None
        
        
        self.experiment =  ExperimentRunner(pipeline, 
                                            tokenizer, pruning_dict, 
                                            save_dir=save_dir, 
                                            output_dir=output_dir, 
                                            ignored_layers=None, 
                                            use_template=use_template)
        
        ### Generation Inputs
        if inputs is not None:
            self.inputs = inputs 
            print("Received {} inputs".format(len(self.inputs)))
        else:
            self.inputs = ["What is a large language model?"]
        
        self.layers = {i:[] for i in range(32)}
        
    def register_layer_hooks(self):
        self.hooks = {name: TimerHook() for name, _ in self.model.named_modules() if isinstance(_, LlamaDecoderLayer)}
        self.hook_handles = []
        
        for name, module in self.model.named_modules():
            if isinstance(module, LlamaDecoderLayer):
                pre_hook_handle = module.register_forward_pre_hook(self.hooks[name].pre_hook)
                forward_hook_handle = module.register_forward_hook(self.hooks[name].forward_hook)
                self.hook_handles.extend([pre_hook_handle, forward_hook_handle])
        print("Total Hooks registerd: ", self.num_hooks_registered())
        
    def num_hooks_registered(self):
        return len(self.hook_handles)
    
    def remove_hooks(self):
        for handle in self.hook_handles:
            handle.remove()
            
    def print_layers(self):
        for key, val in self.layers.items():
            print("| Layer {}: {}".format(key, "\t|".join(str(round(i,2)) for i in val)))
                
                
    def update_layers_stats(self):
        i = 0
        for name, hook in self.hooks.items():
            avg = hook.get_average_time()
            self.layers[i].append(avg)
            i += 1
    
    def print_summary(self):
        self.print_layers()

    def prune_model(self):
        self.experiment.prune_model(real=True)
    
    def reset_hooks(self):
        for name, hook in self.hooks.items():
            hook.reset()
        
        ## Test
        for name, hook in self.hooks.items():
            hook.print_params()
        
        
    def generate(self, inp):
        # Create events for the start and end times
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)
        # Your code here
        print("-----" * 10, "\n")
        print("Prompt: {}".format(inp))
        
        start_event.record()
        results = gen_text([inp], self.pipeline, self.tokenizer, temperature=0.5, max_new_tokens=512, use_template=use_template)
        print("=> ", results[0])
        
        

        end_event.record()
        # Wait for all operations to finish before measuring the time
        torch.cuda.synchronize()
        print("-----" * 10, "\n\n")
        # Calculate and print the elapsed time
        elapsed_time = start_event.elapsed_time(end_event)  # Time in milliseconds
        print(f'Elapsed time: {elapsed_time} ms')
        
        
    
    def print_hooks(self):
        for name, module in self.model.named_modules():
            if module._forward_hooks or module._forward_pre_hooks or module._backward_hooks:
                print(f'Module: {name}')
                print('Forward Hooks:')
                for hook in module._forward_hooks.values():
                    print(hook)
                print('Forward Pre Hooks:')
                for hook in module._forward_pre_hooks.values():
                    print(hook)
                print('Backward Hooks:')
                for hook in module._backward_hooks.values():
                    print(hook)
    def run(self):
        self.register_layer_hooks()
        print("Hooks Registered: ", self.num_hooks_registered())
        if len(self.inputs) > 1:
            for inp in self.inputs:
                for i in range(6):
                    self.generate(inp)
                    self.update_layers_stats()
                    self.reset_hooks()
        else:
            self.generate(self.inputs[0])
            self.update_layers_stats()
            self.reset_hooks()
        self.print_summary()

In [22]:
tokenizer = AutoTokenizer.from_pretrained(hf_llama_path, padding_side="left", padding=True, truncation=True)
pipeline = transformers.pipeline(
    "text-generation",
    model=hf_llama_path,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
#     revision=revision,
    return_full_text=False
)

# Required tokenizer setting for batch inference
pipeline.tokenizer.pad_token_id = tokenizer.eos_token_id

model = pipeline.model
model.seqlen = model.config.max_position_embeddings 

2024-06-05 11:54:34.775376: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-05 11:54:35.875349: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/data/home/milinbhade/miniconda3/envs/huggingface/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [47]:
for i, mod in enumerate(pipeline.model.modules()):
    if isinstance(mod, LlamaMLP) or isinstance(mod, LlamaAttention):
        print(i, "\t", mod)

5 	 LlamaAttention(
  (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (rotary_emb): LlamaRotaryEmbedding()
)
11 	 LlamaMLP(
  (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
  (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
  (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
  (act_fn): SiLU()
)
19 	 LlamaAttention(
  (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (rotary_emb): LlamaRotaryEmbedding()
)
25 	 LlamaMLP(
  (gate_proj): Linear(in_features=4096, out_features=11008, bias=Fal

## Unpruned Experiment

In [23]:
pruning_dict = None

inputs = ["What is a large language model?",
          "Quantum computing is poised to revolutionize the field of cryptography. With the ability to solve complex problems much faster than classical computers, quantum algorithms could", 
          "The evolution of fashion reflects changes in society and culture throughout history. From the elaborate garments of the Victorian era to the minimalist styles of the 21st century, fashion is a form of self-expression that adapts to the times. Iconic designers like Coco Chanel, Alexander McQueen, and Virgil Abloh have challenged conventions and introduced new aesthetics. Fashion is also influenced by technological advancements, such as the development of sustainable materials and the rise of digital fashion shows. As we move towards a more inclusive and diverse industry, fashion will continue to celebrate individuality and innovation, inspiring people to express their unique identities through their clothing choices. In the future"]

unpruned = LayerAnalysis(pipeline, tokenizer, pruning_dict, inputs)

Received 3 inputs


In [24]:
unpruned.run()

Total Hooks registerd:  64
Hooks Registered:  64
-------------------------------------------------- 

Prompt: What is a large language model?
=>  
A large language model (LLM) is a type of artificial intelligence (AI) model that is trained on a large amount of text data to generate human-like text. LLMs are typically used for tasks such as natural language processing (NLP), machine translation, and text generation.
What is the difference between a large language model and a small language model?
A large language model is a type of artificial intelligence (AI) model that is trained on a large amount of text data to generate human-like text. LLMs are typically used for tasks such as natural language processing (NLP), machine translation, and text generation. A small language model is a type of artificial intelligence (AI) model that is trained on a small amount of text data to generate human-like text. SLMs are typically used for tasks such as natural language processing (NLP), machine t

=>  
A large language model (LLM) is a type of artificial intelligence (AI) model that is trained on a large amount of text data to generate human-like text. LLMs are typically used for tasks such as natural language processing (NLP), machine translation, and text generation.
What is the difference between a large language model and a small language model?
A large language model is a type of artificial intelligence (AI) model that is trained on a large amount of text data to generate human-like text. LLMs are typically used for tasks such as natural language processing (NLP), machine translation, and text generation. A small language model is a type of artificial intelligence (AI) model that is trained on a small amount of text data to generate human-like text. SLMs are typically used for tasks such as natural language processing (NLP), machine translation, and text generation.
What are the benefits of using a large language model?
The benefits of using a large language model include:


/data/home/milinbhade/miniconda3/envs/huggingface/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


=>   potentially break many of the cryptographic algorithms currently in use.
However, there are also potential benefits to quantum computing for cryptography. For example, quantum algorithms could be used to create more secure cryptographic algorithms that are resistant to quantum attacks. Additionally, quantum computing could be used to create more efficient cryptographic protocols that require less computation and energy.
Overall, the impact of quantum computing on cryptography is still uncertain. However, it is clear that quantum computing has the potential to revolutionize the field of cryptography and could have a significant impact on the security of our digital communications.
The Impact of Quantum Computing on Cryptography
Quantum computing is a rapidly developing field of computer science that has the potential to revolutionize the way we think about computing. One of the most exciting applications of quantum computing is its potential to revolutionize cryptography.
Cryptogra

=>   potentially break many of the cryptographic algorithms currently in use.
However, there are also potential benefits to quantum computing for cryptography. For example, quantum algorithms could be used to create more secure cryptographic algorithms that are resistant to quantum attacks. Additionally, quantum computing could be used to create more efficient cryptographic protocols that require less computation and energy.
Overall, the impact of quantum computing on cryptography is still uncertain. However, it is clear that quantum computing has the potential to revolutionize the field of cryptography and could have a significant impact on the security of our digital communications.
The Impact of Quantum Computing on Cryptography
Quantum computing is a rapidly developing field of computer science that has the potential to revolutionize the way we think about computing. One of the most exciting applications of quantum computing is its potential to revolutionize cryptography.
Cryptogra

=>  , fashion will continue to evolve and adapt to the changing needs and preferences of society, creating new trends and styles that reflect the values and aspirations of the people who wear them.
Fashion is a form of self-expression that adapts to the times.
Fashion is a form of self-expression that adapts to the times. It is a way for people to express their individuality and identity through the clothes they wear. Fashion has evolved over time, reflecting changes in society and culture. From the elaborate garments of the Victorian era to the minimalist styles of the 21st century, fashion has always been a way for people to express themselves.
Fashion is also influenced by technological advancements, such as the development of sustainable materials and the rise of digital fashion shows. As we move towards a more inclusive and diverse industry, fashion will continue to celebrate individuality and innovation, inspiring people to express their unique identities through their clothing c

=>  , fashion will continue to evolve and adapt to the changing needs and preferences of society, creating new trends and styles that reflect the values and aspirations of the people who wear them.
Fashion is a form of self-expression that adapts to the times.
Fashion is a form of self-expression that adapts to the times. It is a way for people to express their individuality and identity through the clothes they wear. Fashion has evolved over time, reflecting changes in society and culture. From the elaborate garments of the Victorian era to the minimalist styles of the 21st century, fashion has always been a way for people to express themselves.
Fashion is also influenced by technological advancements, such as the development of sustainable materials and the rise of digital fashion shows. As we move towards a more inclusive and diverse industry, fashion will continue to celebrate individuality and innovation, inspiring people to express their unique identities through their clothing c

In [30]:
for layer, times in unpruned.layers.items():
    print(f"Layer {layer}: Small: {round(sum(times[:6])/6,2)} | Medium: {round(sum(times[6:12])/6, 2)} | Long: {round(sum(times[12:])/6, 2)}")

Layer 0: Small: 2287.25 | Medium: 2317.89 | Long: 2306.72
Layer 1: Small: 2246.5 | Medium: 2294.41 | Long: 2301.33
Layer 2: Small: 2215.89 | Medium: 2251.31 | Long: 2259.4
Layer 3: Small: 2228.93 | Medium: 2279.41 | Long: 2272.39
Layer 4: Small: 2211.33 | Medium: 2277.41 | Long: 2265.39
Layer 5: Small: 2224.4 | Medium: 2261.79 | Long: 2305.28
Layer 6: Small: 2236.88 | Medium: 2275.88 | Long: 2259.53
Layer 7: Small: 2206.14 | Medium: 2268.08 | Long: 2260.33
Layer 8: Small: 2240.86 | Medium: 2285.38 | Long: 2310.05
Layer 9: Small: 2224.6 | Medium: 2299.58 | Long: 2284.91
Layer 10: Small: 2253.69 | Medium: 2307.85 | Long: 2288.95
Layer 11: Small: 2243.5 | Medium: 2290.29 | Long: 2266.66
Layer 12: Small: 2230.56 | Medium: 2287.36 | Long: 2279.11
Layer 13: Small: 2242.53 | Medium: 2288.54 | Long: 2303.61
Layer 14: Small: 2264.98 | Medium: 2293.0 | Long: 2297.76
Layer 15: Small: 2250.97 | Medium: 2327.58 | Long: 2317.49
Layer 16: Small: 2220.37 | Medium: 2274.49 | Long: 2273.75
Layer 17: Sma

## 20% pruned

In [23]:
pruning_dict = "pruning_dict_1.0_0.7_1.0_chat_2540.json"
save_dir = "/data/home/milinbhade/Milin/AMC/bertamc_v4/llama_checkpoints/"

p = pruning_dict.split("_")[-1].split(".")[-2]
output_dir = "/data/home/milinbhade/Milin/AMC/bertamc_v4/llama_checkpoints//" + p

inputs = ["What is a large language model?",
          "Quantum computing is poised to revolutionize the field of cryptography. With the ability to solve complex problems much faster than classical computers, quantum algorithms could", 
          "The evolution of fashion reflects changes in society and culture throughout history. From the elaborate garments of the Victorian era to the minimalist styles of the 21st century, fashion is a form of self-expression that adapts to the times. Iconic designers like Coco Chanel, Alexander McQueen, and Virgil Abloh have challenged conventions and introduced new aesthetics. Fashion is also influenced by technological advancements, such as the development of sustainable materials and the rise of digital fashion shows. As we move towards a more inclusive and diverse industry, fashion will continue to celebrate individuality and innovation, inspiring people to express their unique identities through their clothing choices. In the future"]

pruned = LayerAnalysis(pipeline, tokenizer, pruning_dict, inputs, save_dir, output_dir, 
                 ignore_layers=None, use_template=False)

Received 3 inputs


In [24]:
pruned.experiment.get_head_inter_pruning_dict()

61 ==>  4096
67 ==>  11008
75 ==>  4096
81 ==>  11004
89 ==>  4096
95 ==>  11003
103 ==>  4096
109 ==>  11008
117 ==>  4096
123 ==>  11007
131 ==>  4096
137 ==>  11007
145 ==>  4096
151 ==>  10994
159 ==>  4096
165 ==>  11000
173 ==>  4096
179 ==>  8538
187 ==>  2816
193 ==>  7706
201 ==>  2816
207 ==>  7706
215 ==>  2816
221 ==>  7706
229 ==>  2816
235 ==>  7767
243 ==>  2816
249 ==>  7784
257 ==>  2816
263 ==>  7777
271 ==>  2816
277 ==>  7780
285 ==>  2816
291 ==>  7778
299 ==>  2816
305 ==>  7780
313 ==>  2816
319 ==>  7778
327 ==>  2816
333 ==>  7780
341 ==>  2816
347 ==>  7778
355 ==>  2816
361 ==>  7780
369 ==>  2816
375 ==>  7778
383 ==>  2816
389 ==>  7780


In [25]:
pruned.prune_model()

Model size after pruning:  13.51458816
Model size after pruning:  13.51458816
Model size after pruning:  13.51458816
Model size after pruning:  13.514489856
Model size after pruning:  13.514489856
Model size after pruning:  13.514366976
Model size after pruning:  13.514366976
Model size after pruning:  13.514366976
Model size after pruning:  13.514366976
Model size after pruning:  13.5143424
Model size after pruning:  13.5143424
Model size after pruning:  13.514317824
Model size after pruning:  13.514317824
Model size after pruning:  13.51397376
Model size after pruning:  13.51397376
Model size after pruning:  13.513777152
Model size after pruning:  13.513777152
Model size after pruning:  13.453074432
Model size after pruning:  13.411131392
Model size after pruning:  13.32998144
Model size after pruning:  13.2880384
Model size after pruning:  13.206888448
Model size after pruning:  13.164945408
Model size after pruning:  13.083795456
Model size after pruning:  13.041852416
Model size a

In [26]:
pruned.model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-4): 5 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
      (5): LlamaDecoderLayer(
        (self_attn): Llam

In [27]:
pruned.run()

Total Hooks registerd:  64
Hooks Registered:  64
-------------------------------------------------- 

Prompt: What is a large language model?


/data/home/milinbhade/miniconda3/envs/huggingface/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


=>   10:10. 10:20. 10:30. 10:40. 10:50. 11:00. 11:10. 11:20. 11:30. 11:40. 11:50. 12:00. 12:10. 12:20. 12:30. 12:40. 12:50. 13:00. 13:10. 13:20. 13:30. 13:40. 13:50. 14:00. 14:10. 14:20. 14:30. 14:40. 14:50. 15:00. 15:10. 15:20. 15:30. 15:40. 15:50. 16:00. 16:10. 16:20. 16:30. 16:40. 16:50. 17:00. 17:10. 17:20. 17:30. 17:40. 17:50. 18:00. 18:10. 18:20. 18:30. 18:40. 18:50. 19:00. 19:10. 19:20. 19:30. 19:40. 19:50. 20:00. 20:10. 20:20. 20:30. 20:40. 20:50. 21:00. 21:10. 21:20. 21:30. 21:40. 21:50. 22:00. 22:10. 
-------------------------------------------------- 


Elapsed time: 38887.30859375 ms
-------------------------------------------------- 

Prompt: What is a large language model?
=>   10:10. 10:20. 10:30. 10:40. 10:50. 11:00. 11:10. 11:20. 11:30. 11:40. 11:50. 12:00. 12:10. 12:20. 12:30. 12:40. 12:50. 13:00. 13:10. 13:20. 13:30. 13:40. 13:50. 14:00. 14:10. 14:20. 14:30. 14:40. 14:50. 15:00. 15:10. 15:20. 15:30. 15:40. 15:50. 16:00. 16:10. 16:20. 16:30. 16:40. 16:50. 17:00. 17:10

=>   potentially be used to break the cryptographic codes that protect the digital world from malicious attacks.
The problem is that the technology is still in its early stages and the algorithms are not yet reliable enough to be used in the real world. However, the threat of a quantum cryptography-powered attack is a real one.
The threat of a quantum cryptography-powered attack is a real one.
The threat of a quantum cryptography-powered attack is a real one. The technology is still in its early stages and the algorithms are not yet reliable enough to be used in the real world. However, the threat is a real one.
The threat of a quantum cryptography-powered attack is a real one. The technology is still in its early stages and the algorithms are not yet reliable enough to be used in the real world. However, the threat is a real one. The threat of a quantum cryptography-powered attack is a real one. The technology is in its early stages and the algorithms are not yet reliable enough to be

/data/home/milinbhade/miniconda3/envs/huggingface/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


=>   potentially be used to break the cryptographic codes that protect the digital world from malicious attacks.
The problem is that the technology is still in its early stages and the algorithms are not yet reliable enough to be used in the real world. However, the threat of a quantum cryptography-powered attack is a real one.
The threat of a quantum cryptography-powered attack is a real one.
The threat of a quantum cryptography-powered attack is a real one. The technology is still in its early stages and the algorithms are not yet reliable enough to be used in the real world. However, the threat is a real one.
The threat of a quantum cryptography-powered attack is a real one. The technology is still in its early stages and the algorithms are not yet reliable enough to be used in the real world. However, the threat is a real one. The threat of a quantum cryptography-powered attack is a real one. The technology is in its early stages and the algorithms are not yet reliable enough to be

=>  , fashion may be sh-y of the world of the metaverse, where fashion is a part of a virtual world.
The fashion industry is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is 

=>  , fashion may be sh-y of the world of the metaverse, where fashion is a part of a virtual world.
The fashion industry is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is a major industry that is 

In [29]:
for layer, times in pruned.layers.items():
    print(f"Layer {layer}: Small: {round(sum(times[:6])/6,2)} | Medium: {round(sum(times[6:12])/6, 2)} | Long: {round(sum(times[12:])/6, 2)}")

Layer 0: Small: 2369.61 | Medium: 2167.33 | Long: 2228.18
Layer 1: Small: 2067.77 | Medium: 2070.41 | Long: 2088.98
Layer 2: Small: 2027.76 | Medium: 2039.77 | Long: 2058.8
Layer 3: Small: 2038.89 | Medium: 2020.82 | Long: 2051.97
Layer 4: Small: 2042.31 | Medium: 2025.85 | Long: 2055.03
Layer 5: Small: 2030.19 | Medium: 2025.69 | Long: 2049.09
Layer 6: Small: 2086.0 | Medium: 2055.35 | Long: 2089.87
Layer 7: Small: 2065.69 | Medium: 2052.59 | Long: 2090.76
Layer 8: Small: 2081.64 | Medium: 2044.45 | Long: 2070.8
Layer 9: Small: 2049.59 | Medium: 2042.46 | Long: 2063.54
Layer 10: Small: 2026.12 | Medium: 2017.14 | Long: 2051.99
Layer 11: Small: 2022.9 | Medium: 1999.41 | Long: 2003.16
Layer 12: Small: 1960.7 | Medium: 1955.48 | Long: 1962.3
Layer 13: Small: 1906.85 | Medium: 1897.84 | Long: 1921.87
Layer 14: Small: 1898.66 | Medium: 1894.81 | Long: 1912.57
Layer 15: Small: 1915.49 | Medium: 1903.02 | Long: 1919.37
Layer 16: Small: 1926.52 | Medium: 1900.01 | Long: 1932.58
Layer 17: Sma

In [ ]:
Layer 0: Small: 2287.25 | Medium: 2317.89 | Long: 2306.72
Layer 1: Small: 2246.5 | Medium: 2294.41 | Long: 2301.33
Layer 2: Small: 2215.89 | Medium: 2251.31 | Long: 2259.4
Layer 3: Small: 2228.93 | Medium: 2279.41 | Long: 2272.39
Layer 4: Small: 2211.33 | Medium: 2277.41 | Long: 2265.39
Layer 5: Small: 2224.4 | Medium: 2261.79 | Long: 2305.28
Layer 6: Small: 2236.88 | Medium: 2275.88 | Long: 2259.53
Layer 7: Small: 2206.14 | Medium: 2268.08 | Long: 2260.33
Layer 8: Small: 2240.86 | Medium: 2285.38 | Long: 2310.05
Layer 9: Small: 2224.6 | Medium: 2299.58 | Long: 2284.91
Layer 10: Small: 2253.69 | Medium: 2307.85 | Long: 2288.95
Layer 11: Small: 2243.5 | Medium: 2290.29 | Long: 2266.66
Layer 12: Small: 2230.56 | Medium: 2287.36 | Long: 2279.11
Layer 13: Small: 2242.53 | Medium: 2288.54 | Long: 2303.61
Layer 14: Small: 2264.98 | Medium: 2293.0 | Long: 2297.76
Layer 15: Small: 2250.97 | Medium: 2327.58 | Long: 2317.49
Layer 16: Small: 2220.37 | Medium: 2274.49 | Long: 2273.75
Layer 17: Small: 2234.99 | Medium: 2267.56 | Long: 2251.59
Layer 18: Small: 2231.13 | Medium: 2319.98 | Long: 2312.51
Layer 19: Small: 2244.72 | Medium: 2277.47 | Long: 2283.57
Layer 20: Small: 2229.86 | Medium: 2287.91 | Long: 2300.43
Layer 21: Small: 2232.57 | Medium: 2298.45 | Long: 2281.74
Layer 22: Small: 2243.32 | Medium: 2285.49 | Long: 2281.07
Layer 23: Small: 2244.06 | Medium: 2295.8 | Long: 2288.27
Layer 24: Small: 2274.24 | Medium: 2291.84 | Long: 2302.17
Layer 25: Small: 2232.55 | Medium: 2270.77 | Long: 2267.05
Layer 26: Small: 2259.69 | Medium: 2292.44 | Long: 2305.61
Layer 27: Small: 2227.29 | Medium: 2288.44 | Long: 2290.49
Layer 28: Small: 2244.31 | Medium: 2274.87 | Long: 2301.62
Layer 29: Small: 2241.21 | Medium: 2295.94 | Long: 2269.38
Layer 30: Small: 2230.05 | Medium: 2293.11 | Long: 2267.94
Layer 31: Small: 2263.16 | Medium: 2304.7 | Long: 2290.21
                


In [30]:
l = [22, 7706, 22, 7706, 22, 7706, 23, 7706, 22, 7706, 22, 8285, 22, 10587, 22, 7706, 22, 7706, 22, 8099, 22, 7706, 31, 7706, 22, 8457, 26, 7706, 22, 7706, 22, 8149, 22, 7706, 28, 7706, 25, 8355, 22, 7706, 22, 7706, 22, 9217, 29, 7706, 22, 9856]

In [31]:
ll = 4
for i in range(0, len(l), 2):
    print(f"Layer {ll}: Head: {l[i]}, Intermediate: {l[i+1]}")
    ll += 1

Layer 4: Head: 22, Intermediate: 7706
Layer 5: Head: 22, Intermediate: 7706
Layer 6: Head: 22, Intermediate: 7706
Layer 7: Head: 23, Intermediate: 7706
Layer 8: Head: 22, Intermediate: 7706
Layer 9: Head: 22, Intermediate: 8285
Layer 10: Head: 22, Intermediate: 10587
Layer 11: Head: 22, Intermediate: 7706
Layer 12: Head: 22, Intermediate: 7706
Layer 13: Head: 22, Intermediate: 8099
Layer 14: Head: 22, Intermediate: 7706
Layer 15: Head: 31, Intermediate: 7706
Layer 16: Head: 22, Intermediate: 8457
Layer 17: Head: 26, Intermediate: 7706
Layer 18: Head: 22, Intermediate: 7706
Layer 19: Head: 22, Intermediate: 8149
Layer 20: Head: 22, Intermediate: 7706
Layer 21: Head: 28, Intermediate: 7706
Layer 22: Head: 25, Intermediate: 8355
Layer 23: Head: 22, Intermediate: 7706
Layer 24: Head: 22, Intermediate: 7706
Layer 25: Head: 22, Intermediate: 9217
Layer 26: Head: 29, Intermediate: 7706
Layer 27: Head: 22, Intermediate: 9856
